<a href="https://colab.research.google.com/github/MichaelGelo/GRP2_CEPARCO_IP/blob/main/CEPARCO_IP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Group 2 - Implementing Hyyrö’s Bit Vector Algorithm Using CUDA SIMT**
## **GROUP 2 - S11**

**MEMBERS:**

- Alfred Bastin S. Agustines
- Allan David C. De Leon
- Michael Angelo Depasucat
- Kai Hiori J. Padilla


##Check if CUDA is present

In [6]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Sep 15 17:01:08 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from google.colab import files

uploaded = files.upload()

for filename, content in uploaded.items():
    with open(filename, "wb") as f:
        f.write(content)

Saving Query_testA3.fasta to Query_testA3.fasta
Saving Query_testCA2.fasta to Query_testCA2.fasta
Saving Query_testA.txt to Query_testA.txt
Saving Query_testC.txt to Query_testC.txt
Saving Query_testCA.fasta to Query_testCA.fasta
Saving Query_testC.fasta to Query_testC.fasta
Saving Query_testA2.fasta to Query_testA2.fasta
Saving Query_testA.fasta to Query_testA.fasta


In [7]:
%%writefile C_utils.h
#ifdef __cplusplus
extern "C" {
#endif

char* read_file_into_string(const char* filename);
char** parse_fasta_file(const char *filename, int *num_sequences);

#ifdef __cplusplus
}
#endif

Writing C_utils.h


In [8]:
%%writefile C_utils.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include "C_utils.h"

#define MAX_LENGTH (1 << 24)
#define MAX_LINE_LENGTH (1 << 14)

char* read_file_into_string(const char* filename) {
    FILE* file = fopen(filename, "rb");
    if (!file) {
        perror("Failed to open file");
        return NULL;
    }
    fseek(file, 0, SEEK_END);
    long file_size = ftell(file);
    rewind(file);
    char* buffer = (char*)malloc(file_size + 1);
    if (!buffer) {
        perror("Failed to allocate memory");
        fclose(file);
        return NULL;
    }
    size_t bytes_read = fread(buffer, 1, file_size, file);
    if (bytes_read != file_size) {
        perror("Failed to read the file completely");
        free(buffer);
        fclose(file);
        return NULL;
    }
    buffer[file_size] = '\0';
    fclose(file);
    return buffer;
}

char** parse_fasta_file(const char *filename, int *num_sequences) {
    FILE *file = fopen(filename, "r");
    if (!file) {
        perror("Failed to open FASTA file");
        return NULL;
    }
    char **sequences = NULL;
    int seq_count = 0;
    char *current_seq = NULL;
    size_t current_seq_len = 0;
    char line[MAX_LINE_LENGTH];

    while (fgets(line, sizeof(line), file)) {
        if (line[0] == '>') {
            if (current_seq != NULL) {
                if (current_seq_len > 0) {
                    sequences = (char**)realloc(sequences, (seq_count + 1) * sizeof(char*));
                    sequences[seq_count] = (char*)malloc(current_seq_len + 1);
                    memcpy(sequences[seq_count], current_seq, current_seq_len);
                    sequences[seq_count][current_seq_len] = '\0';
                    seq_count++;
                }
                free(current_seq);
                current_seq = NULL;
                current_seq_len = 0;
            }
        } else {
            size_t line_len = strlen(line);
            while (line_len > 0 && (line[line_len - 1] == '\n' || line[line_len - 1] == '\r')) {
                line_len--;
            }
            if (current_seq_len + line_len > MAX_LENGTH) {
                line_len = MAX_LENGTH - current_seq_len;
            }
            if (line_len > 0) {
                current_seq = (char*)realloc(current_seq, current_seq_len + line_len + 1);
                memcpy(current_seq + current_seq_len, line, line_len);
                current_seq_len += line_len;
                current_seq[current_seq_len] = '\0';
            }
        }
    }
    if (current_seq != NULL && current_seq_len > 0) {
        sequences = (char**)realloc(sequences, (seq_count + 1) * sizeof(char*));
        sequences[seq_count] = (char*)malloc(current_seq_len + 1);
        memcpy(sequences[seq_count], current_seq, current_seq_len);
        sequences[seq_count][current_seq_len] = '\0';
        seq_count++;
        free(current_seq);
    }
    fclose(file);
    *num_sequences = seq_count;
    return sequences;
}


Writing C_utils.c


##C - Split reference code

In [ ]:
%%writefile C_partition.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdint.h>
#include <time.h>
#include "C_utils.h"

#define reference_file "Reference_100k.fasta"
#define query_file "Query_test.txt"

void write_fasta(FILE *file, const char *sequence, int line_width){
    int length = strlen(sequence);
    for (int i = 0; i < length; i += line_width) {
        fprintf(file, "%.*s\n", line_width, sequence + i);
    }
}

int main() {
    int num_references = 0;
    char **reference_seqs = parse_fasta_file(reference_file, &num_references);

    const char *query = read_file_into_string(query_file);
    int q_length = strlen(query);
    int p = 10700;
    int reference_length = 0;

    FILE *output_file = fopen("partitioned_output.fasta", "w");
    if (output_file == NULL) {
        perror("Unable to open output file");
        return 1;
    }

    for (int i = 0; i < num_references; i++) {
        char *sequence = reference_seqs[i];
        reference_length = strlen(sequence);

        int previous_k = 0;
        int temp = 0;

        while (previous_k < reference_length) {
            int chunk_size = p;
            if (previous_k + p > reference_length) {
                chunk_size = reference_length - previous_k;
            }

            int extended_chunk_size = chunk_size + (q_length - 1);
            if (previous_k + extended_chunk_size > reference_length) {
                extended_chunk_size = reference_length - previous_k;
            }

            char ref_toparti[extended_chunk_size + 1];

            int k;
            for (k = 0; k < extended_chunk_size; k++) {
                ref_toparti[k] = sequence[previous_k + k];
            }
            ref_toparti[k] = '\0';

            // print to terminal
            printf("Chunk from sequence %d: %s\n", temp, ref_toparti);
            temp++;
            // write to output FASTA
            fprintf(output_file, ">%d_%d\n", temp, previous_k);
            write_fasta(output_file, ref_toparti, 60);

            previous_k += p;
        }
    }

    fclose(output_file);
    printf("Partitioned FASTA file written.\n");

    // free reference_seqs if needed (depends on how parse_fasta_file works)
    // free_reference_seqs(reference_seqs, num_references);

    return 0;
}


Overwriting C_partition.c


In [ ]:
%%shell
gcc C_partition.c C_utils.c -o C_partition

In [ ]:
%%shell
./C_partition

Chunk from sequence 0: GTCTGGTTGAGTGCTACAATCGAATATCCAAATTCCTGAGCCTGCTCAATCGTCTGCGCAACGCGCGCCCACTCGACGCTGCTCAGATGCCCATTTGGCTTACCTGCTGCGTCTGGCAGCGCAATGTTTAGATCATAGAACATACTCTCTGCGCTGTGTCTAAATGAATAATGTTAAAATGCCTTGAGGTGTTTATTATTCATATCAGACCATTGAATATTAATATGATCACTAATAGAGCCAGCAACTTTTACAAAGTCTAACTGTGCCAGCTGAAGCTTGGAAATGGGTCCCAGATTAGTAGATCCGCGTGATAAAACGGTGAGCAAAGAATTTGTATTAGGCATGGCCGTCGTTGCCATTAGCAACGTCTATAATTTTGTCGTTGTCTTCACGATCCTGCTCAACTTCCATGTCATCGCCATCCTGGTTGCCGCCGCCATCGTCGTTTTCAGCCTCCGCTCGTGGAAGCGTCTCCAGTATTGTCTCTACGATATGCTCAAGCTCCTCCACGCTGAATCGCTCGTCGCACTCTTCGATAATCAGGTGGAGTTCAACCAGCGACTTTGGCCTGTGATTTATTATCTGCATGACCTCTGCCTTGGTCAGTTCTAGATTGTCAAGTTGCTGCTTTAGCGCTGTGATCTGCTCCACCGACTGCGACGAGCATGGTGTGTTGCTGAGGTACTCCAGCGCTTCAAAGACCAACGTTGTCACATTCTCTGAGTATTTGGCCTGCGTCTGCGGTCGGTATTCCTTGTGTTGTTCGTCCTCTTCGCGAAGCACCAAAAGCGCTTCGTAATTGCTTATAAGTGCTGCCTGCCGATCAACCACTTCCATCGTCAAGGTCTTTGCAAACTGCTCCTATATTTAAGAGCACAGTATGCTATGAGCAAGTGATAACAAAGAAAGTGTAGAATATTGCTACTTTTATGGTTGCGTTGTAATAGGTTTTATTAATGTCCTTGGCTTGTTTT

In [ ]:
from google.colab import files
files.download("partitioned_output.fasta")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

##C - Single Query & Multiple Reference

In [ ]:
%%writefile C_hyyro.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdint.h>
#include <time.h>
#include "C_utils.h"

#define MAX_LENGTH (1 << 24)

#define query_file "Query_test.txt"
//#define reference_file "Reference_500k.fasta"
#define reference_file "partitioned_output.fasta"
#define loope 1000
typedef uint64_t bitvector;

int bit_vector_levenshtein(const char *query, const char *reference, int *zero_indices, int *zero_count, int* lowest) {
    int m = strlen(query);
    int n = strlen(reference);
    if (m > MAX_LENGTH || n > MAX_LENGTH) {
        printf("Error: Strings too long for this implementation!\n");
        return -1;
    }

    bitvector Pv = ~0ULL;
    bitvector Mv = 0;
    bitvector Eq[256] = {0};
    bitvector Ph, Mh, Xv, Xh, Xp;

    for(int i = 0; i < m; i++) {
        Eq[(unsigned char)query[i]] |= (1ULL << i);
    }

    int score = m;
    *zero_count = 0;
	  *lowest = m;

    for(int j = 0; j < n; j++) {

        Xv = Eq[(unsigned char)reference[j]] | Mv;
        Xh = ((~Xh & Xv) << 1) & Xp;

        Xh = Xh | ((Xv & Pv) + Pv) ^ Pv | Xv | Mv;
        Ph = Mv | ~(Xh | Pv);
        Mh = Xh & Pv;
        Xp = Xv;

        if (Ph & (1ULL << (m - 1))) score++;
        if (Mh & (1ULL << (m - 1))) score--;

        if (score <= *lowest) *lowest = score;

        Xv = (Ph << 1);
        Pv = (Mh << 1) | ~(Xh | Xv);
        Mv = Xh & Xv;

        if (score == 0) {
            zero_indices[*zero_count] = j;
            (*zero_count)++;
        }
    }

    return score;
}

int main() {
    clock_t start, end;
    double elapsed_total = 0.0;

    double avg_time = 0.0;
    // Read & Print Query
    const char *query = read_file_into_string(query_file);
    printf("Query Sequence:\n%s\n\n", query);

    // Read Reference
    int num_references = 0;
    char **reference_seqs = parse_fasta_file(reference_file, &num_references);

    // Print Reference
    printf("Reference Sequences:\n");
    for (int i = 0; i < num_references; i++) {
        printf("Reference #%d:\n%s\n\n", i + 1, reference_seqs[i]);
    }
    double reference_times[num_references];
    // Process each ref
    for (int i = 0; i < num_references; i++) {
        int distance = 0;
        int zero_count = 0;
        int lowest_score = 0;
        // Alloc
        int *zero_indices = (int*)malloc(strlen(reference_seqs[i]) * sizeof(int));
        if (!zero_indices) {
            perror("Failed to allocate memory for zero_indices");
            continue;
        }
        // Time
        elapsed_total = 0;
        for (int j = 0; j < loope; j++) {
            start = clock();
            distance = bit_vector_levenshtein(query, reference_seqs[i], zero_indices, &zero_count, &lowest_score);
            end = clock();
            elapsed_total += ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC;
        }
        avg_time = elapsed_total/loope;
        reference_times[i] = avg_time;

        // print result lng
        printf("Results for Reference #%d:\n", i + 1);
        printf("Distance: %d, Lowest Score: %d, Hits (zero score positions): %d\n",
               distance, lowest_score, zero_count);
        printf("Zero Indices: ");
        for (int k = 0; k < zero_count; k++) {
            printf("%d ", zero_indices[k]);
        }
        printf("\n\n");

        free(zero_indices);
    }
    double total_time = 0.0;
  for(int f = 0; f < num_references; f++){
      total_time += reference_times[f];
      printf("\nTime for reference %d is %f milliseconds\n", f+1, reference_times[f]);
  }
    printf("\nTotal time: %f milliseconds", total_time);

    // Free mem
    free((void*)query);
    for (int i = 0; i < num_references; i++) {
        free(reference_seqs[i]);
    }
    free(reference_seqs);

    return 0;
}


Overwriting C_hyyro.c


In [ ]:
%%shell
gcc C_hyyro.c C_utils.c -o C_hyyro

In [ ]:
%%shell
./C_hyyro

Query Sequence:
GTCTGGTTGAGTGCTACAATCGAATATCCAAATTCCTGAGCCTGCTCAATCGTCTGCGCA

Reference Sequences:
Reference #1:
...

Reference #2:
...

Reference #3:
...

Results for Reference #1:
Distance: 31, Lowest Score: 0, Hits (zero score positions): 158
Zero Indices: 59 1119 2179 3239 4299 5359 6419 7479 8539 9599 10659 11719 12779 13839 14899 15959 17019 18079 19139 20199 21259 22319 23379 24439 25499 26559 27619 28679 29739 30799 31859 32919 33979 35039 36099 37159 38219 39279 40339 41399 42459 43519 44579 45639 46699 47759 48819 49879 50939 51999 53059 54119 55179 56239 57299 58359 59419 60479 61539 62599 63659 64719 65779 66839 67899 68959 70019 71079 72139 73199 74259 75319 76379 77439 78499 79559 80619 81679 82739 83799 84859 85919 86979 88039 89099 90159 91219 92279 93339 94399 95695 96755 97815 98875 99935 100995 102055 103115 104175 105235 106295 107355 108415 109475 110535 111595 112655 113715 114775 115835 116895 117955 119015 120075 121135 122195 123255 124315 125375 126435 127495 

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>
#include "C_utils.h"

// Prefetch + page creation + memadvise
#define MAX_LENGTH (1 << 24)
#define MAX_HITS 1024

// params to change here ↓↓↓

#define query_file "Query_1.txt"
#define reference_file "Reference_87.fasta"
#define threadsPerBlock 256
#define loope 10

// params to change here ↑↑↑

typedef uint64_t bitvector;
__constant__ bitvector d_Eq[256];

// Bit-vector Myers alignment function
template<typename CharPtr>
__host__ __device__ int bit_vector_levenshtein(
    int query_length,
    const CharPtr reference,
    int reference_length,
    const bitvector *Eq,
    int *zero_indices,
    int *zero_count,
    int* lowest)
{
    bitvector Pv = ~0ULL;
    bitvector Mv = 0;
    bitvector Ph = 0;
    bitvector Mh = 0;
    bitvector Xv = 0;
    bitvector Xh = 0;
    bitvector Xp = 0;
    int score = query_length;

    *zero_count = 0;
    *lowest = score;

    for (int j = 0; j < reference_length; j++) {
        unsigned char c = reference[j];

        Xv = Eq[c] | Mv;
        Xh = ((~Xh & Xv) << 1) & Xp;

        Xh = Xh | ((((Xv & Pv) + Pv) ^ Pv) | Xv | Mv);
        Ph = Mv | ~(Xh | Pv);

        Mh = Xh & Pv;
        Xp = Xv;

        if (Ph & (1ULL << (query_length - 1))) score++;
        if (Mh & (1ULL << (query_length - 1))) score--;
        if (score <= *lowest) *lowest = score;

        if (score == 0) {
            zero_indices[*zero_count] = j;
            (*zero_count)++;
        }

        Xv = (Ph << 1);
        Pv = (Mh << 1) | ~(Xh | Xv);
        Mv = Xh & Xv;
    }
    return score;
}

__global__ void levenshtein_kernel(
    int query_length,
    const char *__restrict__ references,
    const int *__restrict__ reference_lengths,
    int *__restrict__ distances,
    int *__restrict__ zero_counts,
    int *__restrict__ zero_indices,
    int num_references,
    int *__restrict__ d_lowest)
{
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx >= num_references) return;

    const char *reference = &references[idx * MAX_LENGTH];
    int reference_length = reference_lengths[idx];
    int *current_zero_indices = &zero_indices[idx * MAX_HITS];

    distances[idx] = bit_vector_levenshtein(
        query_length,
        reference,
        reference_length,
        d_Eq,
        current_zero_indices,
        &zero_counts[idx],
        &d_lowest[idx]
    );
}

int main(){
    // Read Query
    char* query = (char*)read_file_into_string(query_file);
    int query_length = strlen(query);

    // Read Reference
    int num_references = 0;
    char **reference_seqs = parse_fasta_file(reference_file, &num_references);
    int *reference_lengths = (int*)malloc(num_references * sizeof(int));
    for (int i = 0; i < num_references; i++) {
        reference_lengths[i] = strlen(reference_seqs[i]);
    }
    int ref_len = reference_lengths[0]; // use first reference length for throughput calc

    // Precompute Eq
    bitvector h_Eq[256] = {0};
    for (int i = 0; i < query_length; i++) {
        h_Eq[(unsigned char)query[i]] |= (1ULL << i);
    }
    cudaMemcpyToSymbol(d_Eq, h_Eq, sizeof(bitvector) * 256);

    // Allocate Unified Memory
    char *references;
    int *d_reference_lengths, *d_distances, *d_zero_indices, *d_zero_counts, *d_lowest;
    cudaMallocManaged(&references, num_references * MAX_LENGTH * sizeof(char));
    cudaMallocManaged(&d_reference_lengths, num_references * sizeof(int));
    cudaMallocManaged(&d_distances, num_references * sizeof(int));
    cudaMallocManaged(&d_zero_indices, num_references * MAX_HITS * sizeof(int));
    cudaMallocManaged(&d_zero_counts, num_references * sizeof(int));
    cudaMallocManaged(&d_lowest, num_references * sizeof(int));

    // Copy references
    for (int i = 0; i < num_references; i++) {
        strncpy(&references[i * MAX_LENGTH], reference_seqs[i], MAX_LENGTH);
        d_reference_lengths[i] = reference_lengths[i];
    }

    // Prefetch to GPU
    int device = -1; cudaGetDevice(&device);
    cudaMemAdvise(references, num_references * MAX_LENGTH * sizeof(char), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_reference_lengths, num_references * sizeof(int), cudaMemAdviseSetReadMostly, device);
    cudaMemPrefetchAsync(references, num_references * MAX_LENGTH * sizeof(char), device, NULL);
    cudaMemPrefetchAsync(d_reference_lengths, num_references * sizeof(int), device, NULL);

    int blocksPerGrid = (num_references + threadsPerBlock - 1) / threadsPerBlock;

    // CUDA event timing
    cudaEvent_t ev_start, ev_stop;
    cudaEventCreate(&ev_start);
    cudaEventCreate(&ev_stop);

    // Warm-up
    levenshtein_kernel<<<blocksPerGrid, threadsPerBlock>>>(
        query_length, references, d_reference_lengths,
        d_distances, d_zero_counts, d_zero_indices,
        num_references, d_lowest
    );
    cudaDeviceSynchronize();

    // Record start
    cudaEventRecord(ev_start);

    // Timed launches
    for (int i = 0; i < loope; ++i) {
        levenshtein_kernel<<<blocksPerGrid, threadsPerBlock>>>(
            query_length, references, d_reference_lengths,
            d_distances, d_zero_counts, d_zero_indices,
            num_references, d_lowest
        );
    }
    cudaDeviceSynchronize();

    // Record stop
    cudaEventRecord(ev_stop);
    cudaEventSynchronize(ev_stop);

    // Compute elapsed time and throughput
    float ms;
    cudaEventElapsedTime(&ms, ev_start, ev_stop);
    double T = ms / 1000.0; // seconds
    double total_bit_ops = double(ref_len) * 23.0;
    double ops_per_sec = total_bit_ops / T;
    double bases_per_sec = double(ref_len) / T;

    printf("Pure kernel time over %d runs: %.3f ms\n", loope, ms);
    printf("OPS/sec: %.3e ops/sec\n", ops_per_sec);
    printf("Bases/sec: %.3e bases/sec\n", bases_per_sec);

    // Cleanup (post-processing outside timed region)
    cudaFree(references);
    cudaFree(d_reference_lengths);
    cudaFree(d_distances);
    cudaFree(d_zero_counts);
    cudaFree(d_zero_indices);
    cudaFree(d_lowest);
    for (int i = 0; i < num_references; i++) free(reference_seqs[i]);
    free(reference_seqs);
    free(reference_lengths);
    free(query);

    return 0;
}


##C - Multiple Query & Multiple Reference

In [ ]:
%%writefile C_hyyro_multiple.c

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdint.h>
#include <time.h>
#include "C_utils.h"

#define MAX_LENGTH (1 << 24)

#define query_file "Query_test.fasta"
#define reference_file "Reference_87.fasta"
#define loope 10

typedef uint64_t bitvector;

int bit_vector_levenshtein(const char *query, const char *reference, int *zero_indices, int *zero_count, int* lowest) {
    int m = strlen(query);
    int n = strlen(reference);
    if (m > MAX_LENGTH || n > MAX_LENGTH) {
        printf("Error: Strings too long for this implementation!\n");
        return -1;
    }

    bitvector Pv = ~0ULL;
    bitvector Mv = 0;
    bitvector Eq[256] = {0};
    bitvector Ph, Mh, Xv, Xh, Xp;

    for(int i = 0; i < m; i++) {
        Eq[(unsigned char)query[i]] |= (1ULL << i);
    }

    int score = m;
    *zero_count = 0;
	  *lowest = m;

    for(int j = 0; j < n; j++) {

        Xv = Eq[(unsigned char)reference[j]] | Mv;
        Xh = ((~Xh & Xv) << 1) & Xp;

        Xh = Xh | ((Xv & Pv) + Pv) ^ Pv | Xv | Mv;
        Ph = Mv | ~(Xh | Pv);

        Mh = Xh & Pv;
        Xp = Xv;

        if (Ph & (1ULL << (m - 1))) score++;
        if (Mh & (1ULL << (m - 1))) score--;
		    if (score <= *lowest) *lowest = score;

        Xv = (Ph << 1);
        Pv = (Mh << 1) | ~(Xh | Xv);
        Mv = Xh & Xv;


        if (score == 0) {
            zero_indices[*zero_count] = j;
            (*zero_count)++;
        }
    }
    return score;
}

int main() {
    clock_t start, end;
    double elapsed_total = 0.0;

    int num_queries = 0, num_references = 0;
    // Read & Print Query
    char **query_seqs = parse_fasta_file(query_file, &num_queries);
    for (int i = 0; i < num_queries; i++) {
        printf("Query #%d:\n%s\n\n", i + 1, query_seqs[i]);
    }

    // Read Reference
    char **reference_seqs = parse_fasta_file(reference_file, &num_references);
    for (int i = 0; i < num_references; i++) {
        printf("Reference #%d:\n%s\n\n", i + 1, reference_seqs[i]);
    }

    // Process each ref for each query
    for (int q = 0; q < num_queries; q++) {
        for (int r = 0; r < num_references; r++) {
            int distance = 0, zero_count = 0, lowest_score = 0;
            int *zero_indices = (int*)malloc(strlen(reference_seqs[r]) * sizeof(int));
            if (!zero_indices) {
                perror("Failed to allocate memory for zero_indices");
                continue;
            }
            // Time with loope
            for (int j = 0; j < loope; j++) {
            start = clock();
            distance = bit_vector_levenshtein(query_seqs[q], reference_seqs[r], zero_indices, &zero_count, &lowest_score);
            end = clock();
            elapsed_total += ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC;
            }

            printf("Query #%d vs Reference #%d:\n", q + 1, r + 1);
            printf("Distance: %d, Lowest Score: %d, Hits (zero score positions): %d\n", distance, lowest_score, zero_count);
            printf("Zero Indices: ");
            for (int k = 0; k < zero_count; k++) {
                printf("%d ", zero_indices[k]);
            }
            printf("\n\n");

            free(zero_indices);
        }
    }

    printf("Function (C - Multiple Query & Multiple Reference) average time for %u loops is %f milliseconds\n", loope, elapsed_total / loope);

    // Free mem
    for (int i = 0; i < num_queries; i++) free(query_seqs[i]);
    free(query_seqs);
    for (int i = 0; i < num_references; i++) free(reference_seqs[i]);
    free(reference_seqs);

    return 0;
}


Writing C_hyyro_multiple.c


In [ ]:
%%shell
gcc C_hyyro_multiple.c -o C_hyyro_multiple C_utils.c

In [ ]:
%%shell
./C_hyyro_multiple

Failed to open FASTA file: No such file or directory
Failed to open FASTA file: No such file or directory
Function (C - Multiple Query & Multiple Reference) average time for 10 loops is 0.000000 milliseconds


##CUDA - Single Query & Multiple Reference

In [ ]:
%%writefile CUDA.cu

#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>
#include "C_utils.h"

// Prefetch + page creation + memadvise
#define MAX_LENGTH (1 << 24)
#define MAX_HITS 1024

// params to change here ↓↓↓

#define query_file "Query_testA.txt"
#define reference_file "Query_testCA.fasta"
#define threadsPerBlock 256
#define loope 50

// params to change here ↑↑↑


typedef uint64_t bitvector;
__constant__ bitvector d_Eq[256];

__host__ __device__ int bit_vector_levenshtein(int query_length, const char *reference, int reference_length, const bitvector *Eq, int *zero_indices, int *zero_count, int* lowest) {
    if (query_length > MAX_LENGTH || reference_length > MAX_LENGTH) {
        return -1;
    }

    bitvector Pv = ~0ULL;
    bitvector Mv = 0;
    bitvector Ph = 0;
    bitvector Mh = 0;
    bitvector Xv = 0;
    bitvector Xh = 0;
    bitvector Xp = 0;
    int score = query_length;


    *zero_count = 0;
    *lowest = score;

    for (int j = 0; j < reference_length; j++) {
        unsigned char c = reference[j];

        Xv = Eq[c] | Mv;
        Xh = ((~Xh & Xv) << 1) & Xp;

        Xh = Xh | ((((Xv & Pv) + Pv) ^ Pv) | Xv | Mv);
        Ph = Mv | ~(Xh | Pv);

        Mh = Xh & Pv;
        Xp = Xv;

        if (Ph & (1ULL << (query_length - 1))) score++;
        if (Mh & (1ULL << (query_length - 1))) score--;
        if (score <= *lowest) *lowest = score;

        Xv = (Ph << 1);
        Pv = (Mh << 1) | ~(Xh | Xv);
        Mv = Xh & Xv;

        if (score == 0) {
            zero_indices[*zero_count] = j;
            (*zero_count)++;
        }
    }
    return score;
}

__global__ void levenshtein_kernel(int query_length, const char *__restrict__ references, const int *__restrict__ reference_lengths, int *__restrict__ distances,
                                   int *__restrict__ zero_counts, int *__restrict__ zero_indices, int num_references, int *__restrict__ d_lowest) {

    int idx = blockIdx.x * blockDim.x + threadIdx.x;

    if (idx < num_references) {
        const char *reference = &references[idx * MAX_LENGTH];
        int reference_length = reference_lengths[idx];
        int *current_zero_indices = &zero_indices[idx * MAX_HITS];

        distances[idx] = bit_vector_levenshtein(
            query_length,
            reference,
            reference_length,
            d_Eq,
            current_zero_indices,
            &zero_counts[idx],
            &d_lowest[idx]
        );
    }
}

int main(){
    // Read Query
    const char* query = read_file_into_string(query_file);
    int query_length = strlen(query);

    // Read Reference
    int num_references = 0;
    char **reference_seqs = parse_fasta_file(reference_file, &num_references);
    int *reference_lengths = (int*)malloc(num_references * sizeof(int));
    if (!reference_seqs) {
        free((void*)query);
        return 1;
    }

    for (int i = 0; i < num_references; i++) {
        reference_lengths[i] = strlen(reference_seqs[i]);
    }


    // Precompute Eq for the query and store it in host memory.
    bitvector h_Eq[256] = {0};
    for (int i = 0; i < query_length; i++) {
        h_Eq[(unsigned char)query[i]] |= (1ULL << i);
    }

    // Copy Eq to constant memory on the device.
    cudaMemcpyToSymbol(d_Eq, h_Eq, sizeof(bitvector) * 256);

    // Allocate Unified Memory
    char *references;
    int *d_reference_lengths, *d_distances, *d_zero_indices, *d_zero_counts, *d_lowest;
    cudaMallocManaged(&references, num_references * MAX_LENGTH * sizeof(char));
    cudaMallocManaged(&d_reference_lengths, num_references * sizeof(int));
    cudaMallocManaged(&d_distances, num_references * sizeof(int));
    cudaMallocManaged(&d_zero_indices, num_references * MAX_HITS * sizeof(int));
    cudaMallocManaged(&d_zero_counts, num_references * sizeof(int));
    cudaMallocManaged(&d_lowest, num_references * sizeof(int));

    // copy ref and ref_length to managed memory
    for (int i = 0; i < num_references; i++) {
        strncpy(&references[i * MAX_LENGTH], reference_seqs[i], MAX_LENGTH);
        d_reference_lengths[i] = reference_lengths[i];
    }

    // get gpu id
    int device = -1;
    cudaGetDevice(&device);


    // memory advise
    cudaMemAdvise(references, num_references * MAX_LENGTH * sizeof(char), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_reference_lengths, num_references * sizeof(int), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_distances, num_references * sizeof(int), cudaMemAdviseSetPreferredLocation, device);
    cudaMemAdvise(d_zero_indices, num_references * MAX_HITS * sizeof(int), cudaMemAdviseSetPreferredLocation, device);
    cudaMemAdvise(d_zero_counts, num_references * sizeof(int), cudaMemAdviseSetPreferredLocation, device);
    cudaMemAdvise(d_lowest, num_references * sizeof(int), cudaMemAdviseSetPreferredLocation, device);


    // prefetch data
    cudaMemPrefetchAsync(references, num_references * MAX_LENGTH * sizeof(char), device, NULL);
    cudaMemPrefetchAsync(d_reference_lengths, num_references * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_distances, num_references * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_zero_indices, num_references * MAX_HITS * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_zero_counts, num_references * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_lowest, num_references * sizeof(int), device, NULL);

    // setup CUDA kernel
    int blocksPerGrid = (num_references + threadsPerBlock - 1) / threadsPerBlock;

    // print infos
    printf("*** function = Levenshtein Distance\n");
    printf("numReferences = %d\n", num_references);
    printf("numBlocks = %d, numThreads = %d\n", blocksPerGrid, threadsPerBlock);





    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);



    // Warm up
    levenshtein_kernel<<<blocksPerGrid, threadsPerBlock>>>(query_length, references, d_reference_lengths, d_distances, d_zero_counts, d_zero_indices, num_references, d_lowest);
    cudaDeviceSynchronize();

    cudaEventRecord(start);

    // Execute kernel multiple times
    for (size_t i = 0; i < loope; i++) {
        levenshtein_kernel<<<blocksPerGrid, threadsPerBlock>>>(query_length, references, d_reference_lengths, d_distances, d_zero_counts, d_zero_indices, num_references, d_lowest);
    }

    // barrier
    cudaDeviceSynchronize();

    // Stop timing
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);

    float ms;
    cudaEventElapsedTime(&ms, start, stop);

    float avg_ms = ms / loope;
    printf("Average kernel time: %.3f ms/run\n", avg_ms);

    // Prefetch distances back to CPU
    cudaMemPrefetchAsync(d_distances, num_references * sizeof(int), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(d_zero_counts, num_references * sizeof(int), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(d_zero_indices, num_references * MAX_HITS * sizeof(int), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(d_lowest, num_references * sizeof(int), cudaCpuDeviceId, NULL);
    cudaDeviceSynchronize();

    // error checking
    size_t err_count = 0;
    for (int i = 0; i < num_references; i++) {
        int expected_zero_indices[MAX_HITS] = {0};
        int expected_zero_count = 0;
        int expected_lowest = 0;
        int expected_distance = bit_vector_levenshtein(query_length, reference_seqs[i], reference_lengths[i],
                                                       h_Eq, expected_zero_indices, &expected_zero_count, &expected_lowest);


        if (d_distances[i] != expected_distance || d_zero_counts[i] != expected_zero_count || d_lowest[i] != expected_lowest) {
            err_count++;
            printf("Mismatch in result for reference %d:\n", i);
            printf(" - Expected distance: %d, CUDA distance: %d\n", expected_distance, d_distances[i]);
            printf(" - Expected zero count: %d, CUDA zero count: %d\n", expected_zero_count, d_zero_counts[i]);
            printf(" - Expected lowest: %d, CUDA lowest: %d\n", expected_lowest, d_lowest[i]);


            printf(" - Expected zero indices: ");
            for (int j = 0; j < expected_zero_count; j++) {
                printf("%d ", expected_zero_indices[j]);
            }
            printf("\n - CUDA zero indices: ");
            for (int j = 0; j < d_zero_counts[i]; j++) {
                printf("%d ", d_zero_indices[i * MAX_HITS + j]);
            }
            printf("\n");
        }
    }

    printf("Query Sequence:\n%s\n\n", query);

    printf("Reference Sequences:\n");
    for (int i = 0; i < num_references; i++) {
        printf("Reference #%d:\n%s\n\n", i + 1, reference_seqs[i]);
    }


    // Print results
    for (int i = 0; i < num_references; i++) {
        printf("Results for Reference #%d:\n", i + 1);
        printf("Distance: %d, Lowest: %d, Hits (zero score positions): %d\n",
              d_distances[i], d_lowest[i], d_zero_counts[i]);
        printf("Zero Indices: ");
        for (int k = 0; k < d_zero_counts[i]; k++) {
            printf("%d ", d_zero_indices[i * MAX_HITS + k]);
        }
        printf("\n\n");
    }

    printf("Error count (CUDA program): %zu\n\n", err_count);

    // Free memory
    cudaFree(references);
    cudaFree(d_reference_lengths);
    cudaFree(d_distances);
    cudaFree(d_zero_counts);
    cudaFree(d_zero_indices);
    cudaFree(d_lowest);

    for (int i = 0; i < num_references; i++) {
      free((void*)reference_seqs[i]);
    }
    free(reference_seqs);
    free(reference_lengths);

    return 0;
}


Writing CUDA.cu


In [ ]:
%%shell
nvcc -o CUDA CUDA.cu C_utils.c -arch=sm_75

In [ ]:
%%shell
nvprof ./CUDA

==5134== NVPROF is profiling process 5134, command: ./CUDA
*** function = Levenshtein Distance
numReferences = 1
numBlocks = 1, numThreads = 256
Average kernel time: 0.009 ms/run
Query Sequence:
AAAAAAAAA

Reference Sequences:
Reference #1:
CCCCCCCCCAAAAAAAAA

Results for Reference #1:
Distance: 0, Lowest: 0, Hits (zero score positions): 1
Zero Indices: 17 

Error count (CUDA program): 0

==5134== Profiling application: ./CUDA
==5134== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.82%  450.90us        51  8.8410us  7.1040us  93.086us  levenshtein_kernel(int, char const *, int const *, int*, int*, int*, int, int*)
                    0.18%     800ns         1     800ns     800ns     800ns  [CUDA memcpy HtoD]
      API calls:   76.72%  84.312ms         1  84.312ms  84.312ms  84.312ms  cudaMemcpyToSymbol
                   18.78%  20.633ms         6  3.4389ms  2.7320us  20.548ms  cudaMallocManaged
                

## CUDA - Multiple Query & Multiple Reference

In [ ]:
%%writefile CUDA_Multiple.cu

#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>
#include "C_utils.h"

// Prefetch + page creation + memadvise
#define MAX_LENGTH (1 << 24)
#define MAX_HITS 1024

// params to change here ↓↓↓

#define query_file "Query_testA.fasta"
#define reference_file "Query_testCA.fasta"
#define threadsPerBlock 256
#define loope 10

// params to change here ↑↑↑


typedef uint64_t bitvector;
__constant__ bitvector d_Eq[256];

__host__ __device__ int bit_vector_levenshtein(int query_length, const char *reference, int reference_length, const bitvector *Eq, int *zero_indices, int *zero_count, int* lowest) {
    if (query_length > MAX_LENGTH || reference_length > MAX_LENGTH) {
        return -1;
    }

    bitvector Pv = ~0ULL;
    bitvector Mv = 0;
    bitvector Ph = 0;
    bitvector Mh = 0;
    bitvector Xv = 0;
    bitvector Xh = 0;
    bitvector Xp = 0;
    int score = query_length;


    *zero_count = 0;
    *lowest = score;

    for (int j = 0; j < reference_length; j++) {
        unsigned char c = reference[j];

        Xv = Eq[c] | Mv;
        Xh = ((~Xh & Xv) << 1) & Xp;

        Xh = Xh | ((((Xv & Pv) + Pv) ^ Pv) | Xv | Mv);
        Ph = Mv | ~(Xh | Pv);

        Mh = Xh & Pv;
        Xp = Xv;

        if (Ph & (1ULL << (query_length - 1))) score++;
        if (Mh & (1ULL << (query_length - 1))) score--;
        if (score <= *lowest) *lowest = score;

        Xv = (Ph << 1);
        Pv = (Mh << 1) | ~(Xh | Xv);
        Mv = Xh & Xv;

        if (score == 0) {
            zero_indices[*zero_count] = j;
            (*zero_count)++;
        }
    }
    return score;
}

__global__ void levenshtein_kernel(int num_queries, int num_references, const char *__restrict__ d_queries, const int *__restrict__ d_query_lengths, const bitvector *__restrict__ d_Eq_queries,
                                   const char *__restrict__ d_references, const int *__restrict__ d_reference_lengths, int *__restrict__ d_distances, int *__restrict__ d_zero_counts,
    int *__restrict__ d_zero_indices,
    int *__restrict__ d_lowest)
{
    int query_idx = blockIdx.x;
    int reference_idx = blockIdx.y * blockDim.x + threadIdx.x;

    if (query_idx < num_queries && reference_idx < num_references) {
        int query_length = d_query_lengths[query_idx];  // Now matches parameter name

        const char *reference = &d_references[reference_idx * MAX_LENGTH];
        int reference_length = d_reference_lengths[reference_idx];

        const bitvector *Eq = &d_Eq_queries[query_idx * 256];

        int *current_zero_indices = &d_zero_indices[(query_idx * num_references + reference_idx) * MAX_HITS];
        int *current_zero_count = &d_zero_counts[query_idx * num_references + reference_idx];

        d_distances[query_idx * num_references + reference_idx] =
            bit_vector_levenshtein(query_length, reference, reference_length,
                                 Eq, current_zero_indices, current_zero_count,
                                 &d_lowest[query_idx * num_references + reference_idx]);
    }
}

int main(){

    // Read Query
    int num_queries = 0;
    char **query_seqs = parse_fasta_file(query_file, &num_queries);

    // Read Reference
    int num_references = 0;
    char **reference_seqs = parse_fasta_file(reference_file, &num_references);
    if (!reference_seqs) {
        for (int i = 0; i < num_queries; i++) free(query_seqs[i]);
        free(query_seqs);
        return -1;
    }

    int *reference_lengths = (int*)malloc(num_references * sizeof(int));
    for (int i = 0; i < num_references; i++) {
        reference_lengths[i] = strlen(reference_seqs[i]);
    }

    // Precompute Eq for the query and store it in host memory.
    bitvector *h_Eq_queries = (bitvector*)malloc(num_queries * 256 * sizeof(bitvector));
    int *query_lengths = (int*)malloc(num_queries * sizeof(int));
    char **queries_input = (char**)malloc(num_queries * sizeof(char*));

    for (int q = 0; q < num_queries; q++) {
        queries_input[q] = query_seqs[q];
        query_lengths[q] = strlen(queries_input[q]);
        bitvector *h_Eq = &h_Eq_queries[q * 256];
        memset(h_Eq, 0, 256 * sizeof(bitvector));

        for (int i = 0; i < query_lengths[q]; i++) {
            h_Eq[(unsigned char)query_seqs[q][i]] |= (1ULL << i);
        }
    }

    bitvector *d_Eq_queries;
    cudaMalloc(&d_Eq_queries, num_queries * 256 * sizeof(bitvector));
    cudaMemcpy(d_Eq_queries, h_Eq_queries, num_queries * 256 * sizeof(bitvector), cudaMemcpyHostToDevice);

    // Allocate Unified Memory
    char *d_queries, *references;
    int *d_query_lengths, *d_reference_lengths, *d_distances, *d_zero_indices, *d_zero_counts, *d_lowest;

    cudaMallocManaged(&d_queries, num_queries * MAX_LENGTH * sizeof(char));
    cudaMallocManaged(&d_query_lengths, num_queries * sizeof(int));
    cudaMallocManaged(&d_Eq_queries, num_queries * 256 * sizeof(bitvector));

    cudaMallocManaged(&references, num_references * MAX_LENGTH * sizeof(char));
    cudaMallocManaged(&d_reference_lengths, num_references * sizeof(int));
    cudaMallocManaged(&d_distances, num_queries * num_references * sizeof(int)); //
    cudaMallocManaged(&d_zero_counts, num_queries * num_references * sizeof(int)); //
    cudaMallocManaged(&d_zero_indices, num_queries * num_references * MAX_HITS * sizeof(int)); //
    cudaMallocManaged(&d_lowest, num_references * sizeof(int));

    // copy ref and ref_length to managed memory
    for (int i = 0; i < num_references; i++) {
        strncpy(&references[i * MAX_LENGTH], reference_seqs[i], MAX_LENGTH);
        d_reference_lengths[i] = reference_lengths[i];
    }

    for (int q = 0; q < num_queries; q++) {
        strcpy(&d_queries[q * MAX_LENGTH], query_seqs[q]);
        d_query_lengths[q] = strlen(query_seqs[q]);
    }

    cudaMemcpy(d_Eq_queries, h_Eq_queries, num_queries * 256 * sizeof(bitvector), cudaMemcpyHostToDevice);

    // get gpu id
    int device = -1;
    cudaGetDevice(&device);

    // memory advise
    cudaMemAdvise(d_queries, num_queries * MAX_LENGTH * sizeof(char), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_query_lengths, num_queries * sizeof(int), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_Eq_queries, num_queries * 256 * sizeof(bitvector), cudaMemAdviseSetReadMostly, device);

    cudaMemAdvise(references, num_references * MAX_LENGTH * sizeof(char), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_reference_lengths, num_references * sizeof(int), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_distances, num_queries * num_references * sizeof(int), cudaMemAdviseSetPreferredLocation, device); //
    cudaMemAdvise(d_zero_indices, num_queries * num_references * MAX_HITS * sizeof(int), cudaMemAdviseSetPreferredLocation, device); //
    cudaMemAdvise(d_zero_counts, num_queries * num_references * sizeof(int), cudaMemAdviseSetPreferredLocation, device); //
    cudaMemAdvise(d_lowest, num_references * sizeof(int), cudaMemAdviseSetPreferredLocation, device);

    // prefetch data
    cudaMemPrefetchAsync(d_queries, num_queries * MAX_LENGTH * sizeof(char), device, NULL);
    cudaMemPrefetchAsync(d_query_lengths, num_queries * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_Eq_queries, num_queries * 256 * sizeof(bitvector), device, NULL);

    cudaMemPrefetchAsync(references, num_references * MAX_LENGTH * sizeof(char), device, NULL);
    cudaMemPrefetchAsync(d_reference_lengths, num_references * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_distances, num_queries * num_references * sizeof(int), device, NULL); //
    cudaMemPrefetchAsync(d_zero_counts, num_queries * num_references * sizeof(int), device, NULL); //
    cudaMemPrefetchAsync(d_zero_indices, num_queries * num_references * MAX_HITS * sizeof(int), device, NULL); //
    cudaMemPrefetchAsync(d_lowest, num_references * sizeof(int), device, NULL);

    // setup CUDA kernel
    int blocksPerGrid = (num_references + threadsPerBlock - 1) / threadsPerBlock;

    // print infos
    printf("*** function = Levenshtein Distance\n");
    printf("numQueries = %d\n", num_queries);
    printf("numReferences = %d\n", num_references);
    printf("numBlocks = %d, numThreads = %d\n", blocksPerGrid, threadsPerBlock);

    for (int i = 0; i < num_queries; i++) {
        printf("Query #%d:\n%s\n\n", i + 1, query_seqs[i]);
    }
    for (int i = 0; i < num_references; i++) {
        printf("Reference #%d:\n%s\n\n", i + 1, reference_seqs[i]);
    }

    // Execute kernel multiple times
    dim3 gridDim(num_queries); // One block per query
    dim3 blockDim(min(num_references, threadsPerBlock));
    for (size_t i = 0; i < loope; i++) {
        levenshtein_kernel<<<gridDim, blockDim>>>(
        num_queries, num_references,
        d_queries, d_query_lengths, d_Eq_queries,
        references, d_reference_lengths,
        d_distances, d_zero_counts, d_zero_indices, d_lowest);
    }

    // barrier
    cudaDeviceSynchronize();

    // Prefetch distances back to CPU
    cudaMemPrefetchAsync(d_queries, num_queries * MAX_LENGTH * sizeof(char), device, NULL);
    cudaMemPrefetchAsync(d_query_lengths, num_queries * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_Eq_queries, num_queries * 256 * sizeof(bitvector), device, NULL);

    cudaMemPrefetchAsync(references, num_references * MAX_LENGTH * sizeof(char), device, NULL);
    cudaMemPrefetchAsync(d_reference_lengths, num_references * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(d_distances, num_queries * num_references * sizeof(int), device, NULL); //
    cudaMemPrefetchAsync(d_zero_counts, num_queries * num_references * sizeof(int), device, NULL); //
    cudaMemPrefetchAsync(d_zero_indices, num_queries * num_references * MAX_HITS * sizeof(int), device, NULL); //
    cudaMemPrefetchAsync(d_lowest, num_references * sizeof(int), cudaCpuDeviceId, NULL);
    cudaDeviceSynchronize();

    size_t err_count = 0;
    for (int q = 0; q < num_queries; q++) {
        printf("\nResults for Query #%d: \"%s\" (Length: %d)\n", q + 1, query_seqs[q], query_lengths[q]);

        for (int i = 0; i < num_references; i++) {
            int expected_zero_indices[MAX_HITS] = {0};
            int expected_zero_count = 0;
            int expected_lowest = 0;

            int expected_distance = bit_vector_levenshtein(query_lengths[q], reference_seqs[i], reference_lengths[i],
                                                          &h_Eq_queries[q * 256], expected_zero_indices, &expected_zero_count, &expected_lowest);

            if (d_distances[q * num_references + i] != expected_distance || d_zero_counts[q * num_references + i] != expected_zero_count || d_lowest[q * num_references + i] != expected_lowest) {
                err_count++;
                printf("Mismatch in result for reference %d:\n", i);
                printf(" - Expected distance: %d, CUDA distance: %d\n", expected_distance, d_distances[q * num_references + i]);
                printf(" - Expected zero count: %d, CUDA zero count: %d\n", expected_zero_count, d_zero_counts[q * num_references + i]);
                printf(" - Expected lowest: %d, CUDA lowest: %d\n", expected_lowest, d_lowest[q * num_references + i]);
                printf(" - Expected zero indices: ");
                for (int j = 0; j < expected_zero_count; j++) {
                    printf("%d ", expected_zero_indices[j]);
                }
                printf("\n - CUDA zero indices: ");
                for (int j = 0; j < d_zero_counts[q * num_references + i]; j++) {
                    printf("%d ", d_zero_indices[(q * num_references + i) * MAX_HITS + j]);
                }
                printf("\n");
            }
        }

        // Print results
        for (int i = 0; i < num_references; i++) {
            printf("Results for Query #%d & Reference #%d:\n",q+1,  i + 1);
            printf("Distance: %d, Lowest: %d, Hits (zero score positions): %d\n",
                  d_distances[q * num_references + i], d_lowest[q * num_references + i], d_zero_counts[q * num_references + i]);
            printf("Zero Indices: ");
            for (int k = 0; k < d_zero_counts[q * num_references + i]; k++) {
                printf("%d ", d_zero_indices[(q * num_references + i) * MAX_HITS + k]);
            }
            printf("\n\n");
        }
    }

    printf("Error count (CUDA program): %zu\n\n", err_count);

    // Free memory
    cudaFree(references);
    cudaFree(d_reference_lengths);
    cudaFree(d_distances);
    cudaFree(d_zero_counts);
    cudaFree(d_zero_indices);
    cudaFree(d_queries);
    cudaFree(d_query_lengths);
    cudaFree(d_Eq_queries);
    cudaFree(d_lowest);

    for (int i = 0; i < num_queries; i++) {
        free(query_seqs[i]);
    }
    free(query_seqs);
    free(query_lengths);

    for (int i = 0; i < num_references; i++) {
        free(reference_seqs[i]);
    }
    free(reference_seqs);
    free(reference_lengths);

    return 0;
}

Writing CUDA_Multiple.cu


In [ ]:
%%shell
nvcc -o CUDA CUDA_Multiple.cu C_utils.c -arch=sm_75

In [ ]:
%%shell
nvprof ./CUDA

==5279== NVPROF is profiling process 5279, command: ./CUDA
*** function = Levenshtein Distance
numQueries = 1
numReferences = 1
numBlocks = 1, numThreads = 256
Query #1:
AAAAAAAAA

Reference #1:
CCCCCCCCCAAAAAAAAA


Results for Query #1: "AAAAAAAAA" (Length: 9)
Results for Query #1 & Reference #1:
Distance: 0, Lowest: 0, Hits (zero score positions): 1
Zero Indices: 17 

Error count (CUDA program): 0

==5279== Profiling application: ./CUDA
==5279== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   53.82%  213.88us         2  106.94us  1.2480us  212.64us  [CUDA memcpy HtoD]
                   46.18%  183.55us        10  18.354us  7.7110us  113.57us  levenshtein_kernel(int, int, char const *, int const *, unsigned long const *, char const *, int const *, int*, int*, int*, int*)
      API calls:   76.73%  87.417ms         1  87.417ms  87.417ms  87.417ms  cudaMalloc
                   18.13%  20.660ms         9  2.2956ms  

# Thesis

## CUDA parallelized partitioning

In [ ]:
%%writefile C_partition.cu

#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <stdint.h>
#include <cuda_runtime.h>
#include "C_utils.h"

#define reference_file "Reference_500k.fasta"
#define query_file "Query_test.txt"

void write_fasta(FILE *file, const char *sequence, int line_width) {
    int length = strlen(sequence);
    for (int i = 0; i < length; i += line_width) {
        fprintf(file, "%.*s\n", line_width, sequence + i);
    }
}

// CUDA kernel: extract one chunk per block
__global__ void partition_kernel(const char *reference,
                                 int reference_length,
                                 int q_length,
                                 int p,
                                 char *output,
                                 int *chunk_sizes,
                                 int *offsets) {
    int chunk_id = blockIdx.x;
    int start = chunk_id * p;
    if (start >= reference_length) return;

    int chunk_size = p;
    if (start + p > reference_length)
        chunk_size = reference_length - start;

    int extended_chunk_size = chunk_size + (q_length - 1);
    if (start + extended_chunk_size > reference_length)
        extended_chunk_size = reference_length - start;

    // compute where to write this chunk
    int out_offset = offsets[chunk_id];
    for (int k = threadIdx.x; k < extended_chunk_size; k += blockDim.x) {
        output[out_offset + k] = reference[start + k];
    }
    if (threadIdx.x == 0) {
        output[out_offset + extended_chunk_size] = '\0';
        chunk_sizes[chunk_id] = extended_chunk_size;
    }
}

int main() {
    int num_references = 0;
    char **reference_seqs = parse_fasta_file(reference_file, &num_references);

    const char *query = read_file_into_string(query_file);
    int q_length = strlen(query);
    int p = 166700;

    FILE *output_file = fopen("partitioned_output.fasta", "w");
    if (output_file == NULL) {
        perror("Unable to open output file");
        return 1;
    }

    for (int i = 0; i < num_references; i++) {
        char *sequence = reference_seqs[i];
        int reference_length = strlen(sequence);

        // number of chunks
        int num_chunks = (reference_length + p - 1) / p;

        // allocate device memory
        char *d_reference, *d_output;
        int *d_chunk_sizes, *d_offsets;

        // compute offsets for host output
        int *h_chunk_sizes = (int *)malloc(num_chunks * sizeof(int));
        int *h_offsets = (int *)malloc((num_chunks+1) * sizeof(int));
        h_offsets[0] = 0;
        // upper bound: each chunk = p+q
        for (int c = 1; c <= num_chunks; c++) {
            h_offsets[c] = h_offsets[c-1] + (p + q_length);
        }

        size_t total_buf_size = h_offsets[num_chunks];
        char *h_output = (char *)malloc(total_buf_size);

        cudaMalloc(&d_reference, reference_length);
        cudaMalloc(&d_output, total_buf_size);
        cudaMalloc(&d_chunk_sizes, num_chunks * sizeof(int));
        cudaMalloc(&d_offsets, (num_chunks+1) * sizeof(int));

        cudaMemcpy(d_reference, sequence, reference_length, cudaMemcpyHostToDevice);
        cudaMemcpy(d_offsets, h_offsets, (num_chunks+1) * sizeof(int), cudaMemcpyHostToDevice);

        // launch kernel: one block per chunk
        partition_kernel<<<num_chunks, 256>>>(d_reference,
                                             reference_length,
                                             q_length,
                                             p,
                                             d_output,
                                             d_chunk_sizes,
                                             d_offsets);
        cudaDeviceSynchronize();

        // copy back results
        cudaMemcpy(h_output, d_output, total_buf_size, cudaMemcpyDeviceToHost);
        cudaMemcpy(h_chunk_sizes, d_chunk_sizes, num_chunks * sizeof(int), cudaMemcpyDeviceToHost);

        // write results to file
        for (int c = 0; c < num_chunks; c++) {
            char *chunk = h_output + h_offsets[c];
            printf("Chunk %d: %s\n", c, chunk);
            fprintf(output_file, ">%d_%d\n", c, c * p);
            write_fasta(output_file, chunk, 60);
        }

        // free memory
        free(h_output);
        free(h_offsets);
        free(h_chunk_sizes);
        cudaFree(d_reference);
        cudaFree(d_output);
        cudaFree(d_chunk_sizes);
        cudaFree(d_offsets);
    }

    fclose(output_file);
    printf("Partitioned FASTA file written (CUDA).\n");

    return 0;
}


Overwriting C_partition.cu


In [ ]:
%%shell
nvcc -o CUDA C_partition.cu C_utils.c -arch=sm_75

In [ ]:
%%shell
nvprof ./CUDA

Failed to open FASTA file: No such file or directory
Failed to open file: No such file or directory
======== Error: Application received signal 139


CalledProcessError: Command 'nvprof ./CUDA
' returned non-zero exit status 15.

In [ ]:
from google.colab import files
files.download("partitioned_output.fasta")

## testing for 256 length character

In [ ]:
%%writefile CUDA_Multiple.cu


#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>
#include <assert.h>
#include <cuda_runtime.h>
#include "C_utils.h"


// Prefetch + page creation + memadvise
#define MAX_LENGTH (1 << 24)
#define MAX_HITS 1024


// params to change here ↓↓↓
#define query_file "Query_test3.fasta"
#define reference_file "partitioned_output.fasta"
#define threadsPerBlock 256
#define loope 10
// params to change here ↑↑↑


// Hyyrö / multi-word parameters
#define WORD uint64_t
#define WORD_BITS 64
#define MAX_WORDS 4 // supports up to 4*64 = 256-bit patterns
#define MAX_PATTERN_BITS (MAX_WORDS * WORD_BITS)
typedef WORD bitvector;


#define CUDA_CHECK(call) \
  do { \
    cudaError_t e = (call); \
    if (e != cudaSuccess) { \
    fprintf(stderr, "CUDA error %s:%d: %s\n", __FILE__, __LINE__, cudaGetErrorString(e)); \
    exit(1); \
  } \
} while (0)

// Device function: Hyyrö-style multi-word Myers Levenshtein (no transpositions).
// - m: pattern length (<= MAX_PATTERN_BITS)
// - reference: pointer to reference sequence
// - reference_length: length of reference
// - Eq: pointer to Eq table for this query: layout Eq[256 * MAX_WORDS], where for each symbol s,
//        Eq[s * MAX_WORDS + k] is the k-th 64-bit chunk (little-endian: chunk 0 is LSB bits 0..63)
// - zero_indices: output positions (start indices) where edit distance == 0 (capped by MAX_HITS)
// - zero_count: out param
// - lowest: out param holding lowest score seen
// Returns final edit distance after scanning full reference (score w.r.t. pattern end).
__host__ __device__ int bit_vector_levenshtein_hyyro(int m, const char *reference, int reference_length,
                                           const bitvector *Eq, int *zero_indices, int *zero_count, int *lowest)
{
    if (m <= 0) {
        *zero_count = 0; *lowest = 0; return 0;
    }
    if (m > MAX_PATTERN_BITS) {
        *zero_count = 0; *lowest = -1; return -2;
    }

    const int W = (m + WORD_BITS - 1) / WORD_BITS; // number of words used
    WORD Pv[MAX_WORDS];
    WORD Mv[MAX_WORDS];
    WORD Ph[MAX_WORDS];
    WORD Mh[MAX_WORDS];
    WORD Xh[MAX_WORDS];

    // mask for last (most-significant) word
    WORD last_mask;
    if ((m % WORD_BITS) == 0) last_mask = ~((WORD)0);
    else last_mask = (((WORD)1 << (m % WORD_BITS)) - 1ULL);

    // init Pv/Mv and mask top word
    for (int k = 0; k < W; ++k) { Pv[k] = ~((WORD)0); Mv[k] = 0ULL; }
    Pv[W-1] &= last_mask;

    int score = m;
    *zero_count = 0;
    *lowest = score;

    for (int j = 0; j < reference_length; ++j) {
        unsigned char c = (unsigned char)reference[j];

        // Each symbol block uses MAX_WORDS stride (host layout).
        const bitvector *Eq_sym = &Eq[(size_t)c * MAX_WORDS];

        // compute Xh using multiword (Eq & Pv) + Pv with carry propagation
        WORD carry = 0ULL;
        for (int k = 0; k < W; ++k) {
            // mask last word's Eq bits to avoid garbage in unused bits
            WORD Eqk = Eq_sym[k];
            if (k == W-1) Eqk &= last_mask;

            WORD EqPv = Eqk & Pv[k];
            WORD sum = Pv[k] + EqPv + carry;
            WORD carry_out = ((sum < Pv[k]) || (carry && sum == Pv[k])) ? 1ULL : 0ULL;

            Xh[k] = (sum ^ Pv[k]) | Eqk | Mv[k];
            carry = carry_out;
        }

        // Ph and Mh
        for (int k = 0; k < W; ++k) {
            Ph[k] = Mv[k] | ~(Xh[k] | Pv[k]);
            Mh[k] = Pv[k] & Xh[k];
        }

        // update score using top bit (m-1)
        WORD top_mask = (WORD)1 << ((m - 1) % WORD_BITS);
        if (Ph[W-1] & top_mask) score++;
        if (Mh[W-1] & top_mask) score--;
        if (score < *lowest) *lowest = score;

        // left-shift Ph (inject 1 at LSB) across words
        WORD carry_in = 1ULL;
        for (int k = 0; k < W; ++k) {
            WORD new_carry_in = (Ph[k] >> (WORD_BITS - 1)) & 1ULL;
            WORD shifted = (Ph[k] << 1) | carry_in;
            Ph[k] = shifted;
            carry_in = new_carry_in;
        }
        Ph[W-1] &= last_mask;

        // left-shift Mh across words (no injection)
        carry_in = 0ULL;
        for (int k = 0; k < W; ++k) {
            WORD new_carry_in = (Mh[k] >> (WORD_BITS - 1)) & 1ULL;
            WORD shifted = (Mh[k] << 1) | carry_in;
            Mh[k] = shifted;
            carry_in = new_carry_in;
        }
        Mh[W-1] &= last_mask;

        // finalize Pv and Mv, masking last word
        for (int k = 0; k < W; ++k) {
            Pv[k] = (Mh[k] | ~(Xh[k] | Ph[k]));
            Mv[k] = (Ph[k] & Xh[k]);
            if (k == W-1) { Pv[k] &= last_mask; Mv[k] &= last_mask; }
        }

        if (score == 0) {
            int pos = j - m + 1;
            if (pos >= 0 && *zero_count < MAX_HITS) zero_indices[(*zero_count)++] = pos;
        }
    } // end reference loop

    return score;
}

__global__ void levenshtein_kernel(int num_queries, int num_references,
                                   const char *__restrict__ d_queries, const int *__restrict__ d_query_lengths,
                                   const bitvector *__restrict__ d_Eq_queries,
                                   const char *__restrict__ d_references, const int *__restrict__ d_reference_lengths,
                                   int *__restrict__ d_distances, int *__restrict__ d_zero_counts,
                                   int *__restrict__ d_zero_indices, int *__restrict__ d_lowest)
{
    int query_idx = blockIdx.x;
    int reference_idx = blockIdx.y * blockDim.x + threadIdx.x;

    if (query_idx >= num_queries || reference_idx >= num_references) return;

    int query_length = d_query_lengths[query_idx];
    const char *reference = &d_references[(size_t)reference_idx * MAX_LENGTH];
    int reference_length = d_reference_lengths[reference_idx];
    const bitvector *Eq = &d_Eq_queries[(size_t)query_idx * 256 * MAX_WORDS];

    int cell = query_idx * num_references + reference_idx;
    int *current_zero_indices = &d_zero_indices[(size_t)cell * MAX_HITS];
    int *current_zero_count = &d_zero_counts[cell];
    int *current_lowest = &d_lowest[cell];

    // initialize outputs
    *current_zero_count = 0;
    *current_lowest = query_length;

    int dist = bit_vector_levenshtein_hyyro(query_length, reference, reference_length,
                                           Eq, current_zero_indices, current_zero_count, current_lowest);

    d_distances[cell] = dist;
}

// Host helper to build Eq table per query with MAX_WORDS: layout: Eq[s * MAX_WORDS + k], s in [0..255], k in [0..MAX_WORDS-1]
static void build_Eq_table_for_query(const char *query, int qlen, bitvector *hEq /* size 256 * MAX_WORDS */) {
    // zero all entries
    size_t total_entries = (size_t)256 * MAX_WORDS;
    for (size_t i = 0; i < total_entries; ++i) hEq[i] = 0ULL;

    // set bits for each character position
    for (int i = 0; i < qlen; ++i) {
        unsigned char c = (unsigned char)query[i];
        int word = i / WORD_BITS;
        int bit = i % WORD_BITS;
        if (word < MAX_WORDS) { // safety check
            hEq[(size_t)c * MAX_WORDS + word] |= (1ULL << bit);
        }
    }
}

int main() {
    // Read Query
    int num_queries = 0;
    char **query_seqs = parse_fasta_file(query_file, &num_queries);
    if (!query_seqs) {
        fprintf(stderr, "No queries found\n");
        return -1;
    }


    // Read Reference
    int num_references = 0;
    char **reference_seqs = parse_fasta_file(reference_file, &num_references);
    if (!reference_seqs) {
        for (int i = 0; i < num_queries; i++) free(query_seqs[i]);
        free(query_seqs);
        fprintf(stderr, "No references found\n");
        return -1;
    }


    int *reference_lengths = (int*)malloc(num_references * sizeof(int));
    for (int i = 0; i < num_references; i++) {
        reference_lengths[i] = (int)strlen(reference_seqs[i]);
    }


    int *query_lengths = (int*)malloc(num_queries * sizeof(int));
    int max_query_len = 0;
    for (int q = 0; q < num_queries; ++q) {
        query_lengths[q] = (int)strlen(query_seqs[q]);
        if (query_lengths[q] > max_query_len) max_query_len = query_lengths[q];
    }
    if (max_query_len > MAX_PATTERN_BITS) {
        fprintf(stderr, "Error: pattern too long. Max supported = %d bits\n", MAX_PATTERN_BITS);
        return -1;
    }


    // Precompute Eq
    bitvector *h_Eq_queries = (bitvector*)malloc((size_t)num_queries * 256 * MAX_WORDS * sizeof(bitvector));
    for (int q = 0; q < num_queries; q++) {
        build_Eq_table_for_query(query_seqs[q], query_lengths[q],
        &h_Eq_queries[(size_t)q * 256 * MAX_WORDS]);
    }


    // Device allocations (explicit, no unified memory)
    char *d_queries = NULL, *d_references = NULL;
    int *d_query_lengths = NULL, *d_reference_lengths = NULL;
    int *d_distances = NULL, *d_zero_indices = NULL, *d_zero_counts = NULL, *d_lowest = NULL;
    bitvector *d_Eq_queries = NULL;

    // Host buffers for flattening/query/reference and for results
    char *h_queries_flat = (char*)malloc((size_t)num_queries * MAX_LENGTH * sizeof(char));
    char *h_references_flat = (char*)malloc((size_t)num_references * MAX_LENGTH * sizeof(char));
    memset(h_queries_flat, 0, (size_t)num_queries * MAX_LENGTH * sizeof(char));
    memset(h_references_flat, 0, (size_t)num_references * MAX_LENGTH * sizeof(char));

    for (int i = 0; i < num_references; i++) {
        size_t len = strlen(reference_seqs[i]);
        if (len >= MAX_LENGTH) len = MAX_LENGTH - 1;
        memcpy(&h_references_flat[(size_t)i * MAX_LENGTH], reference_seqs[i], len);
        h_references_flat[(size_t)i * MAX_LENGTH + len] = '\0';
    }
    for (int q = 0; q < num_queries; q++) {
        size_t len = strlen(query_seqs[q]);
        if (len >= MAX_LENGTH) len = MAX_LENGTH - 1;
        memcpy(&h_queries_flat[(size_t)q * MAX_LENGTH], query_seqs[q], len);
        h_queries_flat[(size_t)q * MAX_LENGTH + len] = '\0';
    }

    // Allocate device memory
    CUDA_CHECK(cudaMalloc((void**)&d_queries, (size_t)num_queries * MAX_LENGTH * sizeof(char)));
    CUDA_CHECK(cudaMalloc((void**)&d_references, (size_t)num_references * MAX_LENGTH * sizeof(char)));
    CUDA_CHECK(cudaMalloc((void**)&d_query_lengths, num_queries * sizeof(int)));
    CUDA_CHECK(cudaMalloc((void**)&d_reference_lengths, num_references * sizeof(int)));
    CUDA_CHECK(cudaMalloc((void**)&d_Eq_queries, (size_t)num_queries * 256 * MAX_WORDS * sizeof(bitvector)));
    CUDA_CHECK(cudaMalloc((void**)&d_distances, (size_t)num_queries * num_references * sizeof(int)));
    CUDA_CHECK(cudaMalloc((void**)&d_zero_counts, (size_t)num_queries * num_references * sizeof(int)));
    CUDA_CHECK(cudaMalloc((void**)&d_zero_indices, (size_t)num_queries * num_references * MAX_HITS * sizeof(int)));
    CUDA_CHECK(cudaMalloc((void**)&d_lowest, (size_t)num_queries * num_references * sizeof(int)));

    // Copy inputs to device
    CUDA_CHECK(cudaMemcpy(d_queries, h_queries_flat, (size_t)num_queries * MAX_LENGTH * sizeof(char), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_references, h_references_flat, (size_t)num_references * MAX_LENGTH * sizeof(char), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_query_lengths, query_lengths, num_queries * sizeof(int), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_reference_lengths, reference_lengths, num_references * sizeof(int), cudaMemcpyHostToDevice));
    CUDA_CHECK(cudaMemcpy(d_Eq_queries, h_Eq_queries, (size_t)num_queries * 256 * MAX_WORDS * sizeof(bitvector), cudaMemcpyHostToDevice));

    // Kernel launch configuration (keep your style)
    int blocksPerGridY = (num_references + threadsPerBlock - 1) / threadsPerBlock;
    dim3 gridDim(num_queries, blocksPerGridY); // grid.x = queries, grid.y = reference blocks
    dim3 blockDim(threadsPerBlock);

    // Execute kernel multiple times
    for (size_t i = 0; i < loope; i++) {
        levenshtein_kernel<<<gridDim, blockDim>>>(
            num_queries, num_references,
            d_queries, d_query_lengths, d_Eq_queries,
            d_references, d_reference_lengths,
            d_distances, d_zero_counts, d_zero_indices, d_lowest);
    }
    CUDA_CHECK(cudaGetLastError());
    CUDA_CHECK(cudaDeviceSynchronize());

    // Copy results back to host for validation/printing
    int *h_distances = (int*)malloc((size_t)num_queries * num_references * sizeof(int));
    int *h_zero_counts = (int*)malloc((size_t)num_queries * num_references * sizeof(int));
    int *h_zero_indices = (int*)malloc((size_t)num_queries * num_references * MAX_HITS * sizeof(int));
    int *h_lowest = (int*)malloc((size_t)num_queries * num_references * sizeof(int));
    CUDA_CHECK(cudaMemcpy(h_distances, d_distances, (size_t)num_queries * num_references * sizeof(int), cudaMemcpyDeviceToHost));
    CUDA_CHECK(cudaMemcpy(h_zero_counts, d_zero_counts, (size_t)num_queries * num_references * sizeof(int), cudaMemcpyDeviceToHost));
    CUDA_CHECK(cudaMemcpy(h_zero_indices, d_zero_indices, (size_t)num_queries * num_references * MAX_HITS * sizeof(int), cudaMemcpyDeviceToHost));
    CUDA_CHECK(cudaMemcpy(h_lowest, d_lowest, (size_t)num_queries * num_references * sizeof(int), cudaMemcpyDeviceToHost));

    // Validation and results
    size_t err_count = 0;
    for (int q = 0; q < num_queries; q++) {
        printf("\nResults for Query #%d: \"%s\" (Length: %d)\n", q + 1, query_seqs[q], query_lengths[q]);

        for (int i = 0; i < num_references; i++) {
            int expected_zero_indices[MAX_HITS] = {0};
            int expected_zero_count = 0;
            int expected_lowest = 0;

            // Run CPU validation using the same multi-word function
            int expected_distance = bit_vector_levenshtein_hyyro(query_lengths[q], reference_seqs[i], reference_lengths[i],
                                                               &h_Eq_queries[(size_t)q * 256 * MAX_WORDS],
                                                               expected_zero_indices, &expected_zero_count, &expected_lowest);

            int cell = q * num_references + i;
            if (h_distances[cell] != expected_distance ||
                h_zero_counts[cell] != expected_zero_count ||
                h_lowest[cell] != expected_lowest) {
                err_count++;
                printf("Mismatch in result for reference %d:\n", i + 1);
                printf(" - Expected distance: %d, CUDA distance: %d\n", expected_distance, h_distances[cell]);
                printf(" - Expected zero count: %d, CUDA zero count: %d\n", expected_zero_count, h_zero_counts[cell]);
                printf(" - Expected lowest: %d, CUDA lowest: %d\n", expected_lowest, h_lowest[cell]);
            }

            // Print results
            printf("Results for Query #%d & Reference #%d:\n", q + 1, i + 1);
            printf("Distance: %d, Lowest: %d, Hits (zero score positions): %d\n",
                  h_distances[cell], h_lowest[cell], h_zero_counts[cell]);
            if (h_zero_counts[cell] > 0) {
                printf("Zero Indices: ");
                for (int k = 0; k < h_zero_counts[cell]; k++) {
                    printf("%d ", h_zero_indices[cell * MAX_HITS + k]);
                }
                printf("\n");
            }
            printf("\n");
        }
    }

    printf("Error count (CUDA program): %zu\n\n", err_count);

    // Free device memory
    CUDA_CHECK(cudaFree(d_references));
    CUDA_CHECK(cudaFree(d_reference_lengths));
    CUDA_CHECK(cudaFree(d_distances));
    CUDA_CHECK(cudaFree(d_zero_counts));
    CUDA_CHECK(cudaFree(d_zero_indices));
    CUDA_CHECK(cudaFree(d_queries));
    CUDA_CHECK(cudaFree(d_query_lengths));
    CUDA_CHECK(cudaFree(d_Eq_queries));
    CUDA_CHECK(cudaFree(d_lowest));

    // Free host memory
    free(h_queries_flat);
    free(h_references_flat);
    free(h_Eq_queries);
    free(h_distances);
    free(h_zero_counts);
    free(h_zero_indices);
    free(h_lowest);

    free(query_lengths);
    for (int i = 0; i < num_queries; i++) free(query_seqs[i]);
    free(query_seqs);
    for (int i = 0; i < num_references; i++) free(reference_seqs[i]);
    free(reference_seqs);
    free(reference_lengths);

    return 0;
}


Writing CUDA_Multiple.cu


In [ ]:
%%shell
nvcc -o CUDA CUDA_Multiple.cu C_utils.c -arch=sm_75

In [ ]:
%%shell
nvprof ./CUDA

==14459== NVPROF is profiling process 14459, command: ./CUDA

Results for Query #1: "GTCTGGTTGAGTGCTACAATCGAATATCCAAATTCCTGAGCCTGCTCAATCGTCTGCGCAACGTGTCTGGTTGAGTGCTACAATCGAATATCCAAATTCCTGAGCCTGCTCAATCGTCTGCGCAACGT" (Length: 128)
Results for Query #1 & Reference #1:
Distance: 10635, Lowest: 30, Hits (zero score positions): 0

Results for Query #1 & Reference #2:
Distance: 10635, Lowest: 61, Hits (zero score positions): 0

Results for Query #1 & Reference #3:
Distance: 10635, Lowest: 63, Hits (zero score positions): 0

Results for Query #1 & Reference #4:
Distance: 10635, Lowest: 60, Hits (zero score positions): 0

Results for Query #1 & Reference #5:
Distance: 10635, Lowest: 61, Hits (zero score positions): 0

Results for Query #1 & Reference #6:
Distance: 10635, Lowest: 66, Hits (zero score positions): 0

Results for Query #1 & Reference #7:
Distance: 10635, Lowest: 59, Hits (zero score positions): 0

Results for Query #1 & Reference #8:
Distance: 10635, Lowest: 64, Hits (zero score po

## test 2

In [ ]:
%%writefile CUDA_Multiple2.cu
// CUDA_Multiple2.cu
#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>
#include <cuda_runtime.h>
#include "C_utils.h"

// Prefetch + page creation + memadvise
#define MAX_LENGTH (1 << 24)  // Support up to 16M characters
#define MAX_HITS 1024
#define MAX_QUERY_WORDS 4

// params to change here ↓↓↓
#define query_file "Query_testA.fasta"
#define reference_file "Query_testCA.fasta"
#define threadsPerBlock 256
#define loope 10
// params to change here ↑↑↑

typedef uint64_t bitvector;
typedef uint64_t WORD;
#define WORD_SIZE 64
#define WORD_BITS 64
#define MAX_WORDS ((MAX_LENGTH + WORD_SIZE - 1) / WORD_SIZE)
#define MAX_PATTERN_BITS MAX_LENGTH

// -----------------------------------------------------------------------------
// Host implementation: same algorithm but using host malloc/free (used for expected)
// -----------------------------------------------------------------------------
int bit_vector_levenshtein_hyyro(int m, const char *reference, int reference_length,
                                 const bitvector *Eq, int max_words_for_eq,
                                 int *zero_indices, int *zero_count, int *lowest)
{
    if (m <= 0) {
        *zero_count = 0; *lowest = 0; return 0;
    }
    if (m > MAX_PATTERN_BITS) {
        *zero_count = 0; *lowest = -1; return -2;
    }

    const int W = (m + WORD_BITS - 1) / WORD_BITS; // number of words used

    WORD *Pv = (WORD*)malloc(W * sizeof(WORD));
    WORD *Mv = (WORD*)malloc(W * sizeof(WORD));
    WORD *Ph = (WORD*)malloc(W * sizeof(WORD));
    WORD *Mh = (WORD*)malloc(W * sizeof(WORD));
    WORD *Xh = (WORD*)malloc(W * sizeof(WORD));

    if (!Pv || !Mv || !Ph || !Mh || !Xh) {
        if (Pv) free(Pv);
        if (Mv) free(Mv);
        if (Ph) free(Ph);
        if (Mh) free(Mh);
        if (Xh) free(Xh);
        *zero_count = 0; *lowest = -1; return -1;
    }

    // mask for last word
    WORD last_mask;
    if ((m % WORD_BITS) == 0) last_mask = ~((WORD)0);
    else last_mask = (((WORD)1 << (m % WORD_BITS)) - 1ULL);

    for (int k = 0; k < W; ++k) { Pv[k] = ~((WORD)0); Mv[k] = 0ULL; }
    Pv[W-1] &= last_mask;

    int score = m;
    *zero_count = 0;
    *lowest = score;

    for (int j = 0; j < reference_length; ++j) {
        unsigned char c = (unsigned char)reference[j];
        const bitvector *Eq_sym = &Eq[(size_t)c * max_words_for_eq];

        WORD carry = 0ULL;
        for (int k = 0; k < W; ++k) {
            WORD Eqk = Eq_sym[k];
            if (k == W-1) Eqk &= last_mask;

            WORD EqPv = Eqk & Pv[k];
            WORD sum = Pv[k] + EqPv + carry;
            WORD carry_out = ((sum < Pv[k]) || (carry && sum == Pv[k])) ? 1ULL : 0ULL;

            Xh[k] = (sum ^ Pv[k]) | Eqk | Mv[k];
            carry = carry_out;
        }

        for (int k = 0; k < W; ++k) {
            Ph[k] = Mv[k] | ~(Xh[k] | Pv[k]);
            Mh[k] = Pv[k] & Xh[k];
        }

        WORD top_mask = (WORD)1 << ((m - 1) % WORD_BITS);
        if (Ph[W-1] & top_mask) score++;
        if (Mh[W-1] & top_mask) score--;
        if (score < *lowest) *lowest = score;

        WORD carry_in = 1ULL;
        for (int k = 0; k < W; ++k) {
            WORD new_carry_in = (Ph[k] >> (WORD_BITS - 1)) & 1ULL;
            WORD shifted = (Ph[k] << 1) | carry_in;
            Ph[k] = shifted;
            carry_in = new_carry_in;
        }
        Ph[W-1] &= last_mask;

        carry_in = 0ULL;
        for (int k = 0; k < W; ++k) {
            WORD new_carry_in = (Mh[k] >> (WORD_BITS - 1)) & 1ULL;
            WORD shifted = (Mh[k] << 1) | carry_in;
            Mh[k] = shifted;
            carry_in = new_carry_in;
        }
        Mh[W-1] &= last_mask;

        for (int k = 0; k < W; ++k) {
            Pv[k] = (Mh[k] | ~(Xh[k] | Ph[k]));
            Mv[k] = (Ph[k] & Xh[k]);
            if (k == W-1) { Pv[k] &= last_mask; Mv[k] &= last_mask; }
        }

        if (score == 0) {
            int pos = j - m + 1;
            if (pos >= 0 && *zero_count < MAX_HITS) zero_indices[(*zero_count)++] = pos;
        }
    }

    int final_score = score;
    free(Pv); free(Mv); free(Ph); free(Mh); free(Xh);
    return final_score;
}

// -----------------------------------------------------------------------------
// Device implementation that uses preallocated workspace provided by kernel.
// No malloc/free on device.
// -----------------------------------------------------------------------------
__device__ int bit_vector_levenshtein_hyyro_dev(
    int m, const char *reference, int reference_length,
    const bitvector *Eq, int max_words_for_eq,
    int *zero_indices, int *zero_count, int *lowest)
{
    if (m <= 0) {
        *zero_count = 0; *lowest = 0; return 0;
    }
    if (m > MAX_PATTERN_BITS) {
        *zero_count = 0; *lowest = -1; return -2;
    }

    const int W = (m + 63) / 64; // how many words this query needs
    if (W > MAX_QUERY_WORDS) return -3;

    // Stack arrays (per-thread, super fast)
    WORD Pv[MAX_QUERY_WORDS], Mv[MAX_QUERY_WORDS];
    WORD Ph[MAX_QUERY_WORDS], Mh[MAX_QUERY_WORDS], Xh[MAX_QUERY_WORDS];

    // init Pv/Mv and mask top word
    WORD last_mask;
    if ((m % WORD_BITS) == 0) last_mask = ~((WORD)0);
    else last_mask = (((WORD)1 << (m % WORD_BITS)) - 1ULL);

    for (int k = 0; k < W; ++k) { Pv[k] = ~((WORD)0); Mv[k] = 0ULL; }
    Pv[W-1] &= last_mask;

    int score = m;
    *zero_count = 0;
    *lowest = score;

    for (int j = 0; j < reference_length; ++j) {
        unsigned char c = (unsigned char)reference[j];
        const bitvector *Eq_sym = &Eq[(size_t)c * max_words_for_eq];

        WORD carry = 0ULL;
        for (int k = 0; k < W; ++k) {
            WORD Eqk = Eq_sym[k];
            if (k == W-1) Eqk &= last_mask;

            WORD EqPv = Eqk & Pv[k];
            WORD sum = Pv[k] + EqPv + carry;
            WORD carry_out = ((sum < Pv[k]) || (carry && sum == Pv[k])) ? 1ULL : 0ULL;

            Xh[k] = (sum ^ Pv[k]) | Eqk | Mv[k];
            carry = carry_out;
        }

        for (int k = 0; k < W; ++k) {
            Ph[k] = Mv[k] | ~(Xh[k] | Pv[k]);
            Mh[k] = Pv[k] & Xh[k];
        }

        WORD top_mask = (WORD)1 << ((m - 1) % WORD_BITS);
        if (Ph[W-1] & top_mask) score++;
        if (Mh[W-1] & top_mask) score--;
        if (score < *lowest) *lowest = score;

        WORD carry_in = 1ULL;
        for (int k = 0; k < W; ++k) {
            WORD new_carry_in = (Ph[k] >> (WORD_BITS - 1)) & 1ULL;
            WORD shifted = (Ph[k] << 1) | carry_in;
            Ph[k] = shifted;
            carry_in = new_carry_in;
        }
        Ph[W-1] &= last_mask;

        carry_in = 0ULL;
        for (int k = 0; k < W; ++k) {
            WORD new_carry_in = (Mh[k] >> (WORD_BITS - 1)) & 1ULL;
            WORD shifted = (Mh[k] << 1) | carry_in;
            Mh[k] = shifted;
            carry_in = new_carry_in;
        }
        Mh[W-1] &= last_mask;

        for (int k = 0; k < W; ++k) {
            Pv[k] = (Mh[k] | ~(Xh[k] | Ph[k]));
            Mv[k] = (Ph[k] & Xh[k]);
            if (k == W-1) { Pv[k] &= last_mask; Mv[k] &= last_mask; }
        }

        if (score == 0) {
            int pos = j - m + 1;
            if (pos >= 0 && *zero_count < MAX_HITS) zero_indices[(*zero_count)++] = pos;
        }
    }

    return score;
}

// -----------------------------------------------------------------------------
// Kernel: each thread handles one (query_idx, reference_idx) pair.
// Uses a single big device workspace that was allocated from host.
// -----------------------------------------------------------------------------
__global__ void levenshtein_kernel(int num_queries, int num_references,
                                   const char *__restrict__ d_queries, const int *__restrict__ d_query_lengths,
                                   const bitvector *__restrict__ d_Eq_queries, int max_words_needed,
                                   const char *__restrict__ d_references, const int *__restrict__ d_reference_lengths,
                                   int *__restrict__ d_distances, int *__restrict__ d_zero_counts,
                                   int *__restrict__ d_zero_indices, int *__restrict__ d_lowest,
                                   WORD *__restrict__ d_workspace, int workspace_stride_words) // stride = max_words_needed * 5
{
    // mapping:
    // gridDim.x = num_queries
    // gridDim.y = blocksPerGrid (over references)
    int query_idx = blockIdx.x;
    int reference_idx = blockIdx.y * blockDim.x + threadIdx.x;

    if (query_idx >= num_queries || reference_idx >= num_references) return;

    // linear tid used to index into workspace
    // tid = blockIdx.x * (gridDim.y * blockDim.x) + blockIdx.y*blockDim.x + threadIdx.x
    int threads_per_query = gridDim.y * blockDim.x;
    size_t tid = (size_t)blockIdx.x * (size_t)threads_per_query + (size_t)blockIdx.y * (size_t)blockDim.x + (size_t)threadIdx.x;

    WORD *thread_ws = &d_workspace[ tid * (size_t)workspace_stride_words ];

    // carve out arrays
    WORD *Pv = thread_ws + 0 * max_words_needed;
    WORD *Mv = thread_ws + 1 * max_words_needed;
    WORD *Ph = thread_ws + 2 * max_words_needed;
    WORD *Mh = thread_ws + 3 * max_words_needed;
    WORD *Xh = thread_ws + 4 * max_words_needed;

    int query_length = d_query_lengths[query_idx];
    const char *reference = &d_references[(size_t)reference_idx * MAX_LENGTH];
    int reference_length = d_reference_lengths[reference_idx];
    const bitvector *Eq = &d_Eq_queries[(size_t)query_idx * 256 * max_words_needed];

    int cell = query_idx * num_references + reference_idx;
    int *current_zero_indices = &d_zero_indices[(size_t)cell * MAX_HITS];
    int *current_zero_count = &d_zero_counts[cell];
    int *current_lowest = &d_lowest[cell];

    *current_zero_count = 0;
    *current_lowest = query_length;

    int dist = bit_vector_levenshtein_hyyro_dev(query_length, reference, reference_length,
                                                Eq, 4,
                                                current_zero_indices, current_zero_count,
                                                &d_lowest[query_idx * num_references + reference_idx]);

    d_distances[cell] = dist;
}

// -----------------------------------------------------------------------------
// Main
// -----------------------------------------------------------------------------
int main() {
    // Read Query
    int num_queries = 0;
    char **query_seqs = parse_fasta_file(query_file, &num_queries);

    // Read Reference
    int num_references = 0;
    char **reference_seqs = parse_fasta_file(reference_file, &num_references);
    if (!reference_seqs) {
        for (int i = 0; i < num_queries; i++) free(query_seqs[i]);
        free(query_seqs);
        return -1;
    }

    int *reference_lengths = (int*)malloc(num_references * sizeof(int));
    for (int i = 0; i < num_references; i++) {
        reference_lengths[i] = strlen(reference_seqs[i]);
    }

    // Determine max query length
    int max_query_length = 0;
    for (int i = 0; i < num_queries; i++) {
        int len = strlen(query_seqs[i]);
        if (len > max_query_length) max_query_length = len;
    }
    int max_words_needed = (max_query_length + WORD_SIZE - 1) / WORD_SIZE;
    if (max_words_needed < 1) max_words_needed = 1;

    // Precompute Eq for queries (host)
    bitvector *h_Eq_queries = (bitvector*)malloc((size_t)num_queries * 256 * max_words_needed * sizeof(bitvector));
    int *query_lengths = (int*)malloc(num_queries * sizeof(int));

    for (int q = 0; q < num_queries; q++) {
        query_lengths[q] = strlen(query_seqs[q]);
        bitvector *h_Eq = &h_Eq_queries[(size_t)q * 256 * max_words_needed];
        memset(h_Eq, 0, 256 * max_words_needed * sizeof(bitvector));
        for (int i = 0; i < query_lengths[q]; i++) {
            int word_idx = i / WORD_SIZE;
            int bit_idx = i % WORD_SIZE;
            h_Eq[(unsigned char)query_seqs[q][i] * max_words_needed + word_idx] |= (1ULL << bit_idx);
        }
    }

    bitvector *d_Eq_queries = NULL;
    cudaError_t cerr;
    cerr = cudaMalloc(&d_Eq_queries, (size_t)num_queries * 256 * max_words_needed * sizeof(bitvector));
    if (cerr != cudaSuccess) { fprintf(stderr, "cudaMalloc d_Eq_queries failed: %s\n", cudaGetErrorString(cerr)); return -1; }
    cerr = cudaMemcpy(d_Eq_queries, h_Eq_queries, (size_t)num_queries * 256 * max_words_needed * sizeof(bitvector), cudaMemcpyHostToDevice);
    if (cerr != cudaSuccess) { fprintf(stderr, "cudaMemcpy d_Eq_queries failed: %s\n", cudaGetErrorString(cerr)); return -1; }

    // Allocate Unified Memory for sequences and outputs
    char *d_queries = NULL;
    char *references = NULL;
    int *d_query_lengths = NULL;
    int *d_reference_lengths = NULL;
    int *d_distances = NULL;
    int *d_zero_indices = NULL;
    int *d_zero_counts = NULL;
    int *d_lowest = NULL;

    cudaMallocManaged(&d_queries, (size_t)num_queries * MAX_LENGTH * sizeof(char));
    cudaMallocManaged(&d_query_lengths, (size_t)num_queries * sizeof(int));

    cudaMallocManaged(&references, (size_t)num_references * MAX_LENGTH * sizeof(char));
    cudaMallocManaged(&d_reference_lengths, (size_t)num_references * sizeof(int));
    cudaMallocManaged(&d_distances, (size_t)num_queries * num_references * sizeof(int));
    cudaMallocManaged(&d_zero_counts, (size_t)num_queries * num_references * sizeof(int));
    cudaMallocManaged(&d_zero_indices, (size_t)num_queries * num_references * MAX_HITS * sizeof(int));
    cudaMallocManaged(&d_lowest, (size_t)num_queries * num_references * sizeof(int));

    // copy data into managed memory
    for (int i = 0; i < num_references; i++) {
        strncpy(&references[(size_t)i * MAX_LENGTH], reference_seqs[i], MAX_LENGTH);
        d_reference_lengths[i] = reference_lengths[i];
    }
    for (int q = 0; q < num_queries; q++) {
        strncpy(&d_queries[(size_t)q * MAX_LENGTH], query_seqs[q], MAX_LENGTH);
        d_query_lengths[q] = query_lengths[q];
    }

    // device id
    int device = -1;
    cudaGetDevice(&device);

    // memory advise/prefetch (optional)
    cudaMemAdvise(d_queries, (size_t)num_queries * MAX_LENGTH * sizeof(char), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_query_lengths, (size_t)num_queries * sizeof(int), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(references, (size_t)num_references * MAX_LENGTH * sizeof(char), cudaMemAdviseSetReadMostly, device);
    cudaMemAdvise(d_reference_lengths, (size_t)num_references * sizeof(int), cudaMemAdviseSetReadMostly, device);

    // Prefetch to device
    cudaMemPrefetchAsync(d_queries, (size_t)num_queries * MAX_LENGTH * sizeof(char), device, NULL);
    cudaMemPrefetchAsync(d_query_lengths, (size_t)num_queries * sizeof(int), device, NULL);
    cudaMemPrefetchAsync(references, (size_t)num_references * MAX_LENGTH * sizeof(char), device, NULL);
    cudaMemPrefetchAsync(d_reference_lengths, (size_t)num_references * sizeof(int), device, NULL);

    // Setup kernel launch geometry
    int blocksPerGrid = (num_references + threadsPerBlock - 1) / threadsPerBlock;
    if (blocksPerGrid < 1) blocksPerGrid = 1;

    printf("*** function = Levenshtein Distance (Corrected Hyyrö Algorithm)\n");
    printf("numQueries = %d\n", num_queries);
    printf("numReferences = %d\n", num_references);
    printf("blocksPerGrid = %d, threadsPerBlock = %d\n", blocksPerGrid, threadsPerBlock);
    printf("MAX_LENGTH = %d, Max words needed = %d\n", MAX_LENGTH, max_words_needed);

    for (int i = 0; i < num_queries; i++) {
        printf("Query #%d (Length: %d):\n%s\n\n", i + 1, query_lengths[i], query_seqs[i]);
    }
    for (int i = 0; i < num_references; i++) {
        printf("Reference #%d (Length: %d):\n%s\n\n", i + 1, reference_lengths[i], reference_seqs[i]);
    }

    // Allocate a single device workspace for all launched threads
    // Per-thread workspace (words): max_words_needed * 5 (Pv, Mv, Ph, Mh, Xh)
    const int per_thread_words = max_words_needed * 5;
    // total threads launched: num_queries * blocksPerGrid * threadsPerBlock
    size_t total_threads = (size_t)num_queries * (size_t)blocksPerGrid * (size_t)threadsPerBlock;
    if (total_threads == 0) total_threads = 1;

    WORD *d_workspace = NULL;
    size_t workspace_words_total = total_threads * (size_t)per_thread_words;
    cerr = cudaMalloc(&d_workspace, workspace_words_total * sizeof(WORD));
    if (cerr != cudaSuccess) { fprintf(stderr, "cudaMalloc d_workspace failed: %s\n", cudaGetErrorString(cerr)); return -1; }

    // Launch kernel: gridDim.x = num_queries, gridDim.y = blocksPerGrid
    dim3 gridDim((unsigned int)num_queries, (unsigned int)blocksPerGrid);
    dim3 blockDim((unsigned int)threadsPerBlock);

    for (size_t iter = 0; iter < loope; ++iter) {
        levenshtein_kernel<<<gridDim, blockDim>>>(
            num_queries, num_references,
            d_queries, d_query_lengths, d_Eq_queries, max_words_needed,
            references, d_reference_lengths,
            d_distances, d_zero_counts, d_zero_indices, d_lowest,
            d_workspace, per_thread_words  // workspace_stride_words = max_words_needed * 5 inside kernel calc uses max_words_needed
        );
    }

    // check kernel errors
    cudaError_t err = cudaGetLastError();
    if (err != cudaSuccess) {
        fprintf(stderr, "Kernel launch error: %s\n", cudaGetErrorString(err));
    }
    cudaDeviceSynchronize();

    // Prefetch results back to CPU (optional)
    cudaMemPrefetchAsync(d_distances, (size_t)num_queries * num_references * sizeof(int), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(d_zero_counts, (size_t)num_queries * num_references * sizeof(int), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(d_zero_indices, (size_t)num_queries * num_references * MAX_HITS * sizeof(int), cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(d_lowest, (size_t)num_queries * num_references * sizeof(int), cudaCpuDeviceId, NULL);
    cudaDeviceSynchronize();

    // Validate against host function and print
    size_t err_count = 0;
    for (int q = 0; q < num_queries; q++) {
        printf("\nResults for Query #%d: \"%s\" (Length: %d)\n", q + 1, query_seqs[q], query_lengths[q]);
        for (int i = 0; i < num_references; i++) {
            int expected_zero_indices[MAX_HITS] = {0};
            int expected_zero_count = 0;
            int expected_lowest = 0;

            int expected_distance = bit_vector_levenshtein_hyyro(query_lengths[q],
                                                                reference_seqs[i], reference_lengths[i],
                                                                &h_Eq_queries[(size_t)q * 256 * max_words_needed],
                                                                max_words_needed,
                                                                expected_zero_indices, &expected_zero_count, &expected_lowest);

            int cuda_distance = d_distances[q * num_references + i];
            int cuda_zero_count = d_zero_counts[q * num_references + i];
            int cuda_lowest = d_lowest[q * num_references + i];

            if (cuda_distance != expected_distance || cuda_zero_count != expected_zero_count || cuda_lowest != expected_lowest) {
                err_count++;
                printf("Mismatch for reference %d:\n", i);
                printf(" - Expected distance: %d, CUDA distance: %d\n", expected_distance, cuda_distance);
                printf(" - Expected zero count: %d, CUDA zero count: %d\n", expected_zero_count, cuda_zero_count);
                printf(" - Expected lowest: %d, CUDA lowest: %d\n", expected_lowest, cuda_lowest);
                printf(" - Expected zero indices: ");
                for (int j = 0; j < expected_zero_count; j++) printf("%d ", expected_zero_indices[j]);
                printf("\n - CUDA zero indices: ");
                for (int j = 0; j < cuda_zero_count; j++) printf("%d ", d_zero_indices[(q * num_references + i) * MAX_HITS + j]);
                printf("\n");
            }

            // Print results
            printf("Results for Query #%d & Reference #%d:\n", q + 1, i + 1);
            printf("Distance: %d, Lowest: %d, Hits: %d\n",
                   d_distances[q * num_references + i],
                   d_lowest[q * num_references + i],
                   d_zero_counts[q * num_references + i]);
            printf("Zero Indices: ");
            for (int k = 0; k < d_zero_counts[q * num_references + i]; k++) {
                printf("%d ", d_zero_indices[(q * num_references + i) * MAX_HITS + k]);
            }
            printf("\n\n");
        }
    }

    printf("Error count (CUDA program): %zu\n\n", err_count);

    // Free device memory
    cudaFree(d_workspace);
    cudaFree(references);
    cudaFree(d_reference_lengths);
    cudaFree(d_distances);
    cudaFree(d_zero_counts);
    cudaFree(d_zero_indices);
    cudaFree(d_queries);
    cudaFree(d_query_lengths);
    cudaFree(d_Eq_queries);
    cudaFree(d_lowest);

    // Free host memory
    free(h_Eq_queries);
    for (int i = 0; i < num_queries; i++) free(query_seqs[i]);
    free(query_seqs);
    free(query_lengths);

    for (int i = 0; i < num_references; i++) free(reference_seqs[i]);
    free(reference_seqs);
    free(reference_lengths);

    return 0;
}


Overwriting CUDA_Multiple2.cu


In [ ]:
%%shell
nvcc -o CUDA CUDA_Multiple2.cu C_utils.c -arch=sm_75

CUDA_Multiple2.cu(256): warning #177-D: variable "Pv" was declared but never referenced
      WORD *Pv = thread_ws + 0 * max_words_needed;
            ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

CUDA_Multiple2.cu(257): warning #177-D: variable "Mv" was declared but never referenced
      WORD *Mv = thread_ws + 1 * max_words_needed;
            ^

CUDA_Multiple2.cu(258): warning #177-D: variable "Ph" was declared but never referenced
      WORD *Ph = thread_ws + 2 * max_words_needed;
            ^

CUDA_Multiple2.cu(259): warning #177-D: variable "Mh" was declared but never referenced
      WORD *Mh = thread_ws + 3 * max_words_needed;
            ^

CUDA_Multiple2.cu(260): warning #177-D: variable "Xh" was declared but never referenced
      WORD *Xh = thread_ws + 4 * max_words_needed;
            ^



In [ ]:
%%shell
nvprof ./CUDA

==2184== NVPROF is profiling process 2184, command: ./CUDA
*** function = Levenshtein Distance (Corrected Hyyrö Algorithm)
numQueries = 1
numReferences = 1
blocksPerGrid = 1, threadsPerBlock = 256
MAX_LENGTH = 16777216, Max words needed = 1
Query #1 (Length: 9):
AAAAAAAAA

Reference #1 (Length: 18):
CCCCCCCCCAAAAAAAAA


Results for Query #1: "AAAAAAAAA" (Length: 9)
Mismatch for reference 0:
 - Expected distance: 9, CUDA distance: 18
 - Expected zero count: 0, CUDA zero count: 0
 - Expected lowest: 9, CUDA lowest: 9
 - Expected zero indices: 
 - CUDA zero indices: 
Results for Query #1 & Reference #1:
Distance: 18, Lowest: 9, Hits: 0
Zero Indices: 

Error count (CUDA program): 1

==2184== Profiling application: ./CUDA
==2184== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   99.73%  454.29us        10  45.429us  30.336us  180.51us  levenshtein_kernel(int, int, char const *, int const *, unsigned long const *, int, cha

##test 3 I beg Please

In [20]:
%%writefile CUDA_Multiple3.cu
// bitvector_256_myers.cu
#include <cuda_runtime.h>
#include <stdint.h>
#include <stdio.h>
#include <string.h>

#define MAX_LENGTH 256
#define BV_WORDS 4   // 4 * 64 = 256
typedef unsigned long long u64;

struct BitVec256 {
    u64 w[BV_WORDS];
};

__device__ inline void bv_zero(BitVec256 &a) {
    #pragma unroll
    for (int i = 0; i < BV_WORDS; ++i) a.w[i] = 0ULL;
}
__device__ inline void bv_set_all(BitVec256 &a) {
    #pragma unroll
    for (int i = 0; i < BV_WORDS; ++i) a.w[i] = ~0ULL;
}
__device__ inline void bv_copy(const BitVec256 &src, BitVec256 &dst) {
    #pragma unroll
    for (int i = 0; i < BV_WORDS; ++i) dst.w[i] = src.w[i];
}
__device__ inline void bv_or(const BitVec256 &a, const BitVec256 &b, BitVec256 &r) {
    #pragma unroll
    for (int i = 0; i < BV_WORDS; ++i) r.w[i] = a.w[i] | b.w[i];
}
__device__ inline void bv_and(const BitVec256 &a, const BitVec256 &b, BitVec256 &r) {
    #pragma unroll
    for (int i = 0; i < BV_WORDS; ++i) r.w[i] = a.w[i] & b.w[i];
}
__device__ inline void bv_xor(const BitVec256 &a, const BitVec256 &b, BitVec256 &r) {
    #pragma unroll
    for (int i = 0; i < BV_WORDS; ++i) r.w[i] = a.w[i] ^ b.w[i];
}
__device__ inline void bv_not(const BitVec256 &a, BitVec256 &r) {
    #pragma unroll
    for (int i = 0; i < BV_WORDS; ++i) r.w[i] = ~a.w[i];
}

// left shift by 1: r = a << 1  (bits move toward higher indices)
__device__ inline void bv_shl1(const BitVec256 &a, BitVec256 &r) {
    u64 carry = 0ULL;
    #pragma unroll
    for (int i = 0; i < BV_WORDS; ++i) {
        u64 new_carry = (a.w[i] >> 63) & 1ULL;
        r.w[i] = (a.w[i] << 1) | carry;
        carry = new_carry;
    }
}

// right shift by 1: r = a >> 1  (bits move toward lower indices)
__device__ inline void bv_shr1(const BitVec256 &a, BitVec256 &r) {
    u64 carry = 0ULL;
    for (int i = BV_WORDS - 1; i >= 0; --i) {
        u64 new_carry = (a.w[i] & 1ULL);
        r.w[i] = (a.w[i] >> 1) | (carry << 63);
        carry = new_carry;
    }
}

// 256-bit add: r = a + b
// returns final carry-out (0 or 1)
__device__ inline unsigned int bv_add(const BitVec256 &a, const BitVec256 &b, BitVec256 &r) {
    unsigned __int128 sum = 0; // will only be used per-word
    u64 carry = 0ULL;
    #pragma unroll
    for (int i = 0; i < BV_WORDS; ++i) {
        unsigned __int128 local = (unsigned __int128)a.w[i] + (unsigned __int128)b.w[i] + (unsigned __int128)carry;
        r.w[i] = (u64)local;
        carry = (u64)(local >> 64);
    }
    return (unsigned int)carry;
}

// r = a + b where b is Pv in the original formula; we sometimes only need r
// Helper: compute ((Xv & Pv) + Pv)  -> use bv_and then bv_add
__device__ inline void bv_and_add(const BitVec256 &Xv, const BitVec256 &Pv, BitVec256 &tmp, BitVec256 &out) {
    bv_and(Xv, Pv, tmp);    // tmp = Xv & Pv
    bv_add(tmp, Pv, out);   // out = tmp + Pv  (carry ignored)
}

// mask bits above m-1 to zero (so bits >= m are cleared)
__device__ inline void bv_mask_top(BitVec256 &v, int m) {
    if (m >= MAX_LENGTH) return; // no masking needed
    int last_word = (m - 1) / 64;
    int last_bit = (m - 1) % 64;
    u64 last_mask = (last_bit == 63) ? ~0ULL : ((1ULL << (last_bit + 1)) - 1ULL);

    #pragma unroll
    for (int i = last_word + 1; i < BV_WORDS; ++i) v.w[i] = 0ULL;
    v.w[last_word] &= last_mask;
}

// test whether bit (m-1) is set in v: returns 0 or non-zero
__device__ inline int bv_test_msb(const BitVec256 &v, int m) {
    int idx = (m - 1) / 64;
    int off = (m - 1) % 64;
    return ( (v.w[idx] >> off) & 1ULL ) ? 1 : 0;
}

// Device kernel: one thread per query-reference pair
__global__ void bit_vector_levenshtein_256_kernel(
    const char *queries_flat,    // concatenated queries
    const char *refs_flat,       // concatenated references
    const int *q_off, const int *r_off,
    const int *q_len, const int *r_len,
    BitVec256 *Eq_table,         // Eq_table[256] per thread? here it's per thread use (we'll index by 256 * tid)
    int *results,
    int *lowest_scores,
    int *zero_counts,
    int *zero_indices,           // flattened: zero_indices[ tid * maxZeros + z ]
    int maxZerosPerPair
) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    // Load lengths
    int m = q_len[tid];
    int n = r_len[tid];

    if (m <= 0) { results[tid] = 0; lowest_scores[tid] = 0; zero_counts[tid] = 0; return; }
    if (m > MAX_LENGTH) { results[tid] = -1; lowest_scores[tid] = -1; zero_counts[tid] = 0; return; }

    const char *query = queries_flat + q_off[tid];
    const char *ref   = refs_flat + r_off[tid];

    // local per-thread vectors
    BitVec256 Pv, Mv, Xv, Xh, Ph, Mh, tmp, addtmp;
    // Eq for each possible byte value - but to avoid huge memory we assume Eq_table is a contiguous block
    BitVec256 *Eq = Eq_table + (size_t)tid * 256; // Eq[256] for this thread

    // init Eq to zero
    for (int c = 0; c < 256; ++c) bv_zero(Eq[c]);

    // Build Eq: for each position i set bit i in Eq[ query[i] ]
    for (int i = 0; i < m; ++i) {
        unsigned char ch = (unsigned char)query[i];
        int word = i / 64;
        int bit  = i % 64;
        Eq[ch].w[word] |= (1ULL << bit);
    }

    // initialize Pv = all ones, Mv = zero
    bv_set_all(Pv);
    bv_zero(Mv);

    // mask Pv bits above m (Pv should have only m meaningful bits set)
    // Actually Pv initial set_all is fine, but masking helps when using ~ operations later
    bv_mask_top(Pv, m);

    int score = m;
    int lowest = m;
    int zeros = 0;

    // loop over reference
    for (int j = 0; j < n; ++j) {
        unsigned char rc = (unsigned char)ref[j];
        // Xv = Eq[rc] | Mv
        bv_or(Eq[rc], Mv, Xv);

        // Xh = (((Xv & Pv) + Pv) ^ Pv) | Xv | Mv
        bv_and_add(Xv, Pv, tmp, addtmp);   // addtmp = (Xv & Pv) + Pv
        bv_xor(addtmp, Pv, Xh);            // Xh = addtmp ^ Pv
        // Xh = Xh | Xv | Mv
        BitVec256 temp_or;
        bv_or(Xh, Xv, temp_or);
        bv_or(temp_or, Mv, Xh);

        // Ph = Mv | ~(Xh | Pv)
        BitVec256 xh_or_pv;
        bv_or(Xh, Pv, xh_or_pv);
        bv_not(xh_or_pv, tmp);      // tmp = ~(Xh | Pv)
        bv_or(Mv, tmp, Ph);

        // Mh = Xh & Pv
        bv_and(Xh, Pv, Mh);

        // Score updates depend on bit (m-1) of Ph and Mh
        if (bv_test_msb(Ph, m)) score++;
        if (bv_test_msb(Mh, m)) score--;

        if (score <= lowest) lowest = score;

        // Xp (not explicitly used later in this variant) - skip

        // Pv = (Mh << 1) | ~(Xh | (Ph << 1))
        BitVec256 Mh_shl, Ph_shl, xh_or_phshl, not_xh_or_phshl;
        bv_shl1(Mh, Mh_shl);
        bv_shl1(Ph, Ph_shl);
        bv_or(Xh, Ph_shl, xh_or_phshl);
        bv_not(xh_or_phshl, not_xh_or_phshl);
        bv_or(Mh_shl, not_xh_or_phshl, Pv);

        // Mv = Xh & (Ph << 1)
        bv_and(Xh, Ph_shl, Mv);

        // mask Pv and Mv beyond m bits to keep high bits clean
        bv_mask_top(Pv, m);
        bv_mask_top(Mv, m);

        if (score == 0 && zeros < maxZerosPerPair) {
            zero_indices[tid * maxZerosPerPair + zeros] = j;
            zeros++;
        }
    }

    results[tid] = score;
    lowest_scores[tid] = lowest;
    zero_counts[tid] = zeros;
}

/* ------------------------------
   Minimal host-side example (one pair)
   ------------------------------ */

int main() {
    // Example: one query and one reference
    const char *h_q = "AAAAAAACCCCCCCCCCAAAAAAAAAACCCCCCCCCCAAAAAAAAAACCCCCCCCCCAAAAAAAAAA";       // length 5
    const char *h_r = "CCCCCCCCCCAAAAAAAAAACCCCCCCCCCAAAAAAAAAACCCCCCCCCCAAAAAAAAAACCCCCCCCCCAAAAAAAAAA";        // length 4

    // Flattened buffers (one each)
    char *d_q, *d_r;
    cudaMalloc(&d_q, strlen(h_q));
    cudaMalloc(&d_r, strlen(h_r));
    cudaMemcpy(d_q, h_q, strlen(h_q), cudaMemcpyHostToDevice);
    cudaMemcpy(d_r, h_r, strlen(h_r), cudaMemcpyHostToDevice);

    int numPairs = 1;
    int h_q_off[1] = {0}, h_r_off[1] = {0};
    int h_q_len[1] = { (int)strlen(h_q) }, h_r_len[1] = { (int)strlen(h_r) };

    int *d_q_off, *d_r_off, *d_q_len, *d_r_len;
    cudaMalloc(&d_q_off, sizeof(int) * numPairs);
    cudaMalloc(&d_r_off, sizeof(int) * numPairs);
    cudaMalloc(&d_q_len, sizeof(int) * numPairs);
    cudaMalloc(&d_r_len, sizeof(int) * numPairs);
    cudaMemcpy(d_q_off, h_q_off, sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_r_off, h_r_off, sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_q_len, h_q_len, sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_r_len, h_r_len, sizeof(int), cudaMemcpyHostToDevice);

    // Eq table per thread: allocate numPairs * 256 BitVec256's
    BitVec256 *d_Eq;
    cudaMalloc(&d_Eq, sizeof(BitVec256) * 256 * numPairs);

    int *d_results, *d_lowest, *d_zero_counts;
    cudaMalloc(&d_results, sizeof(int) * numPairs);
    cudaMalloc(&d_lowest, sizeof(int) * numPairs);
    cudaMalloc(&d_zero_counts, sizeof(int) * numPairs);

    int maxZeros = 16;
    int *d_zero_indices;
    cudaMalloc(&d_zero_indices, sizeof(int) * maxZeros * numPairs);

    // launch
    dim3 blocks(1), threads(1);
    bit_vector_levenshtein_256_kernel<<<blocks, threads>>>(
        d_q, d_r, d_q_off, d_r_off, d_q_len, d_r_len,
        d_Eq, d_results, d_lowest, d_zero_counts, d_zero_indices, maxZeros
    );
    cudaDeviceSynchronize();

    int h_result = 0, h_low = 0, h_zc = 0, h_zidx[16] = {0};
    cudaMemcpy(&h_result, d_results, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&h_low, d_lowest, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(&h_zc, d_zero_counts, sizeof(int), cudaMemcpyDeviceToHost);
    cudaMemcpy(h_zidx, d_zero_indices, sizeof(int) * maxZeros, cudaMemcpyDeviceToHost);

    printf("Distance: %d, Lowest: %d, ZeroCount: %d\n", h_result, h_low, h_zc);
    for (int i = 0; i < h_zc; ++i) printf(" Zero at ref pos %d\n", h_zidx[i]);

    // free
    cudaFree(d_q); cudaFree(d_r);
    cudaFree(d_q_off); cudaFree(d_r_off); cudaFree(d_q_len); cudaFree(d_r_len);
    cudaFree(d_Eq);
    cudaFree(d_results); cudaFree(d_lowest); cudaFree(d_zero_counts);
    cudaFree(d_zero_indices);

    return 0;
}

Overwriting CUDA_Multiple3.cu


In [21]:
%%shell
nvcc -o CUDA CUDA_Multiple3.cu C_utils.c -arch=sm_75

CUDA_Multiple3.cu(68): warning #177-D: variable "sum" was declared but never referenced
      unsigned __int128 sum = 0;
                        ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"



In [22]:
%%shell
nvprof ./CUDA

==3932== NVPROF is profiling process 3932, command: ./CUDA
Distance: 0, Lowest: 0, ZeroCount: 1
 Zero at ref pos 79
==3932== Profiling application: ./CUDA
==3932== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   91.94%  133.63us         1  133.63us  133.63us  133.63us  bit_vector_levenshtein_256_kernel(char const *, char const *, int const *, int const *, int const *, int const *, BitVec256*, int*, int*, int*, int*, int)
                    5.04%  7.3290us         4  1.8320us  1.6640us  2.1440us  [CUDA memcpy DtoH]
                    3.02%  4.3830us         6     730ns     640ns  1.1520us  [CUDA memcpy HtoD]
      API calls:   99.60%  178.31ms        11  16.210ms  2.1950us  178.27ms  cudaMalloc
                    0.09%  155.69us        11  14.153us  4.1060us  100.94us  cudaFree
                    0.08%  139.70us         1  139.70us  139.70us  139.70us  cudaDeviceSynchronize
                    0.08%  134.93us    

##end